# Clustering Techniques

Complete guide to various clustering techniques experimented and used for short text clustering analysis.

In [29]:
'''Kernel Python Version 3.6.10 '''

# Standard libs
import os
import sys
import json
import warnings
import re
import io
from io import StringIO
import inspect
import shutil
import ast
import string
import time
import pickle
import glob
import traceback
import multiprocessing
import requests
import logging
import math
import pytz
from itertools import chain
from string import Template
from datetime import datetime, timedelta
from dateutil import parser
import base64
from collections import defaultdict, Counter, OrderedDict
from contextlib import contextmanager
import unicodedata
from functools import reduce
import itertools
import tempfile
import jsonschema
from typing import Any, Dict, List, Callable, Optional, Tuple, NamedTuple, Union
from functools import wraps

# graph
import networkx as nx

# Required pkgs
import numpy as np
from numpy import array, argmax
import pandas as pd
import ntpath
import tqdm

# General text correction - fit text for you (ftfy) and others
import ftfy
from fuzzywuzzy import fuzz
from wordcloud import WordCloud
from spellchecker import SpellChecker

# imbalanced-learn
from imblearn.over_sampling import SMOTE, SVMSMOTE, ADASYN

# scikit-learn
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, jaccard_score, silhouette_score, homogeneity_score, calinski_harabasz_score
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from sklearn.neighbors import NearestNeighbors, LocalOutlierFactor
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.base import BaseEstimator, TransformerMixin

# scipy
from scipy import spatial, sparse
from scipy.sparse import coo_matrix, vstack, hstack
from scipy.spatial.distance import euclidean, jensenshannon, cosine, cdist
from scipy.io import mmwrite, mmread
from scipy.stats import entropy
from scipy.cluster.hierarchy import dendrogram, ward, fcluster
import scipy.cluster.hierarchy as sch
from scipy.sparse.csr import csr_matrix
from scipy.sparse.lil import lil_matrix
from scipy.sparse.csgraph import connected_components

# sparse_dot_topn: matrix multiplier
from sparse_dot_topn import awesome_cossim_topn
import sparse_dot_topn.sparse_dot_topn as ct

# Gensim
import gensim
from gensim.models import Phrases, Word2Vec, KeyedVectors, FastText, LdaModel
from gensim import utils
from gensim.utils import simple_preprocess
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim.downloader as api
from gensim import models, corpora, similarities

# NLTK
import nltk
#nltk_model_data_path = "/someppath/"
#nltk.data.path.append(nltk_model_data_path)
from nltk import FreqDist, tokenize, sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import stopwords, PlaintextCorpusReader
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import *
from nltk.translate.bleu_score import sentence_bleu
print("NLTK loaded.")

# Spacy
import spacy
# spacy_model_data_path = "/Users/pranjalpathak/opt/anaconda3/envs/Python_3.6/lib/python3.6/site-packages/en_core_web_lg/en_core_web_lg-2.2.5"
nlp = spacy.load('en_core_web_lg')  # disabling: nlp = spacy.load(spacy_data_path, disable=['ner'])
from spacy import displacy
from spacy.matcher import Matcher
from spacy.lang.en import English
print("Spacy loaded.")

# TF & Keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import CustomObjectScope
from keras.utils.np_utils import to_categorical
from keras.engine.topology import Layer
from keras import backend as K
from keras import initializers as initializers, regularizers, constraints, optimizers
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM
# from keras.layers.core import Input, Dense, Activation
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.models import Sequential, Model, load_model
import tensorflow_hub as hub
print("TensorFlow loaded.")

# Pytorch
import torch
from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead
from transformers import pipeline
from transformers import AutoModel
print("PyTorch loaded.")

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly import offline
%matplotlib inline

# Theme settings
pd.set_option("display.max_columns", 80)
sns.set_context('talk')
sns.set(rc={'figure.figsize':(15,10)})
sns.set_style("darkgrid")
warnings.filterwarnings('ignore')

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-9c8_u2yn because the default path (/ms/dist/wmanalytics/PROJ/homedir/1.0/eisdkuqa/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
unable to import 'smart_open.gcs', disabling that module
/ms/dist/python/PROJ/boto/2.46.1/lib/boto/__init__.py:1142: DeprecationWarning: invalid escape sequence \c
  """
/ms/dist/python/PROJ/boto/2.46.1/lib/boto/pyami/config.py:98: DeprecationWarning: invalid escape sequence \s
  match = re.match("^#import[\s\t]*([^\s^\t]*)[\s\t]*$", line)
/ms/dist/python/PROJ/gensim/3.8.1-py37/exec/lib/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections impor

NLTK loaded.
SPACY loaded.


/ms/dist/python/PROJ/protobuf/3.10.0-py37/exec/lib/google/protobuf/descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from google.protobuf.pyext import _message


pyTorch loaded.


## Dataset

In [77]:
df = pd.DataFrame(list(map(lambda x: x.split('\t')[1].strip() if len(x.split('\t')) > 1 else x, corpus.split('\n'))), columns=['TEXT'])
df.head()

,TEXT
0,We have a client that wants to order a Platinu...
1,how do i stop process of spousal continuation ...
2,can reg D forms be sent via US Mail
3,Is qualifying under Reg D only done digitally
4,can you do an ira to roth conversion inkind


## Preprocessing

Preprocessing Methods:

1. Basic cleaning module
2. Spacy cleaning module

In [78]:
"""
------------------------------------------------------------------------------------
1. BASIC CLEANING MODULE (without contextual word replacement)
------------------------------------------------------------------------------------

## Sample execution ##

vocab_list=["Morgan Stanley", "MS", "KORE", "MS@Work", "?", "is"]
resources_dir_path = "/v/region/na/appl/mswm/ainlp/data/ainlp_dev/Pretrained_Models/Resources/"
preprocessText = basic_preprocessing(resources_dir_path, vocab_list)
df[clean_text_col] = preprocessText.clean(df[text_col])
"""

class basic_preprocessing:
    """
    Perform basic pre-processing steps on any textual data. To add customization, pass a list of keyowrds which needs to 
    be excluded from lemmatization or stop-word removal. Use  'vocab_list'  param to pass a customized list of keywords!
    """

    def __init__(self, resources_dir_path, vocab_list=[], custom=False):
        self.stopwords_file = os.path.join(resources_dir_path, "stopwords.txt")
        self.special_stopwords_file = os.path.join(resources_dir_path, "special_stopwords.txt")
        self.special_characters_file = os.path.join(resources_dir_path, "special_characters.txt")
        self.contractions_file = os.path.join(resources_dir_path, "contractions.json")
        self.chatwords_file = os.path.join(resources_dir_path, "chatwords.txt")
        self.emoticons_file = os.path.join(resources_dir_path, "emoticons.json")
        self.greeting_file = os.path.join(resources_dir_path, "greeting_words.txt")
        self.signature_file = os.path.join(resources_dir_path, "signature_words.txt")
        self.vocab_list = vocab_list
        self.custom = custom
        self.load_resources()
        return

    def load_resources(self):
        # stopwords, special_stopwords, special_characters
        with open(self.stopwords_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.stopwords = [x.rstrip() for x in f.readlines()]
        with open(self.special_stopwords_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.stopwords.extend([x.rstrip() for x in f.readlines()])
        with open(self.special_characters_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.stopwords.extend([x.rstrip() for x in f.readlines()])    
        self.stopwords = sorted(set(self.stopwords))

        # Vocabulary Dictionary (**custom**)
        # - for marking keywords to retain during pre-processing step
        # - modify spacy's stopword checker
        self.vocab_list += ["Morgan Stanley", "MS", "MSO", "MS@Work"]
        self.vocab_list = list(map(str.lower, self.vocab_list))
        self.vocab_list = sorted(set(self.vocab_list))
        # modify stop_words list
        self.stopwords = set(self.stopwords).difference(self.vocab_list)
        # custom regex using these vocab
        self.vocab_dict = {w: "_".join(w.split()) for w in self.vocab_list}
        self.regex_custom = re.compile('|'.join(sorted(map(re.escape, self.vocab_dict), key=len, reverse=True)))
        # for not_stopword in self.vocab_dict.values():
        #     nlp.vocab[not_stopword].is_stop = False
   
        # contractions
        with open(self.contractions_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.contractions = dict(json.load(f))
        # chat-words
        with open(self.chatwords_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.chat_words_map_dict, self.chat_words_list = {}, []
            chat_words = [x.rstrip() for x in f.readlines()]
            for line in chat_words:
                cw = line.split("=")[0]
                cw_expanded = line.split("=")[1]
                self.chat_words_list.append(cw)
                self.chat_words_map_dict[cw] = cw_expanded
            self.chat_words_list = set(self.chat_words_list)
        # emoticons
        with open(self.emoticons_file, "r") as f:
            self.emoticons = re.compile(u'(' + u'|'.join(k for k in json.load(f)) + u')')
        # emojis
        self.emojis = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
        # greetings
        with open(self.greeting_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.greeting_words = [x.rstrip() for x in f.readlines()]
        # signature
        with open(self.signature_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.signature_words = [x.rstrip() for x in f.readlines()]
        # spell-corrector
        self.spell_checker = SpellChecker()   
        return

    
    def custom_cleaning(self, text, reset=False):
        """ *** Perform custom cleaning here... *** """
        # use global variable "regex_custom" in this...
        key = "_"
        text = str(text).strip().lower()
        if reset is False:
            # custom replacement
            text = re.sub(r"directed[\s\,]*share[\s\,]*program(?=[\s\(\[]*dsp[\s\)\]]*)", "", text.lower(), re.IGNORECASE)
            # compile using a dict of words and their expansions, and sub them if found!
            match_and_sub = self.regex_custom.sub(lambda x: self.vocab_dict[x.string[x.start():x.end()]], text)
            return re.sub(r"([\s\n\t\r]+)", " ", match_and_sub).strip()
        else:
            # reverse the change! - use this at the end of preprocessing
            text = text.replace(key, " ")
            return re.sub(r"([\s\n\t\r]+)", " ", text).strip()


    def clean(self, input_sentences):
        cleaned_sentences = []
        for sent in input_sentences:

            # normalize text to "utf-8" encoding
            sent = unicodedata.normalize('NFKD', str(sent)).encode('ascii', 'ignore').decode('utf-8', 'ignore')
       
            # lowercasing
            sent = str(sent).strip().lower()

            # <---------- CUSTOM CLEANING -------------->
            #
            # Mark imp keywords such as: Domain specific, Question words(wh-words), etc, using "vocabulary".
            # Create a "vocab_dict" first.  This dict shall be used to join these keywords (i.e. join them using "_" ), 
            # during pre-processing step, and later un-joined.
            if self.custom:
                sent = self.custom_cleaning(sent, reset=False)
            #
            # <---------- CUSTOM CLEANING -------------->

            # remove Emojis 🔥🔥
            sent = self.emojis.sub(r'', sent)

            # remove emoticons
            sent = self.emoticons.sub(r'', sent)

            # remove common chat-words
            sent = " ".join([self.chat_words_map_dict[w.upper()] if w.upper() in self.chat_words_list else w for w in sent.split()])

            # FIX text
            sent = ftfy.fix_text(sent)

            # Normalize accented chars
            sent = unicodedata.normalize('NFKD', sent).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            
            # Removing <…> web scrape tags
            sent = re.sub(r"\<(.*?)\>", " ", sent)
            
            # Expanding contractions using contractions_file
            sent = re.sub(r"(\w+\'\w+)", lambda x: self.contractions.get(x.group().lower(), x.group().lower()), sent)
            
            # Removing web urls
            sent = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0–9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»""'']))''', " ", sent)
            
            # Removing date formats
            sent = re.sub(r"(\d{4}\-\d{2}\-\d{2}\s\d{2}\:\d{2}\:\d{2}\s\:)", " ", sent)

            # removing punctuations
            # - disable them, when sentence structure needs to be retained.
            sent = re.sub(r"[\$|\#\@\*\%]+\d+[\$|\#\@\*\%]+", " ", sent)
            sent = re.sub(r"\'s", " \'s", sent)
            sent = re.sub(r"\'ve", " \'ve", sent)
            sent = re.sub(r"n\'t", " n\'t", sent)
            sent = re.sub(r"\'re", " \'re", sent)
            sent = re.sub(r"\'d", " \'d", sent)
            sent = re.sub(r"\'ll", " \'ll", sent)
            sent = re.sub(r"[\/,\@,\#,\\,\{,\},\(,\),\[,\],\$,\%,\^,\&,\*,\<,\>]", " ", sent)
            sent = re.sub(r"[\,,\;,\:,\-]", " ", sent)      # main puncts
            # sent = re.sub(r"[\!,\?,\.]", " ", sent)       # sentence delimitters

            # keep only text & numbers
            # - enable them, when only text and numbers matter
            # sent = re.sub(r"\s+", " ", re.sub(r"[\\|\/|\||\{|\}|\[|\]\(|\)]+", " ", re.sub(r"[^A-z0-9]", " ", str(sent))))
            
            # correct spelling mistakes
            # - enable them when english spelling mistakes matter
            # sent = " ".join([self.spell_checker.correction(w) if w in self.spell_checker.unknown(sent.split()) else w for w in sent.split()])
            
            # remove 'modifed' self.stop_words
            sent = " ".join(token.text for token in nlp(sent) if token.text not in self.stopwords)
            
            # lemmatize
            sent = " ".join(token.lemma_ for token in nlp(sent) if token.text not in self.stopwords)
            
            # Removing extra whitespaces
            sent = re.sub(r"([\s\n\t\r]+)", " ", sent).strip()

            # <---------- CUSTOM CLEANING -------------->
            #
            # revers the custom cleaning step!
            if self.custom:
                sent = self.custom_cleaning(sent, reset=True)
            #
            # <---------- CUSTOM CLEANING -------------->

            cleaned_sentences.append(sent.strip().lower())
        return cleaned_sentences

In [79]:
"""
------------------------------------------------------------------------------------
2. SPACY CLEANING MODULE
------------------------------------------------------------------------------------

## Sample execution ##

preprocessText = spacy_preprocessing(resources_dir_path)
process_results = preprocessText.run_pipeline(df[__text_col__].tolist(), operations=["basic","deep","spacy"])
df[__clean_text_col__] = [" ".join(x["lemma_list"]).lower() for x in process_results]
"""

class spacy_preprocessing:
    """
    Performs Spacy cleaning, and genrate spacy text features such as pos, tokens, ner, dependency, etc.
    """
    def __init__(self, resources_dir_path):
        self.stopwords_file = os.path.join(resources_dir_path, "stopwords.txt")
        self.special_stopwords_file = os.path.join(resources_dir_path, "special_stopwords.txt")
        self.special_characters_file = os.path.join(resources_dir_path, "special_characters.txt")
        self.synonym_file = os.path.join(resources_dir_path, "synonyms_noun_verb.txt")
        self.contractions_file = os.path.join(resources_dir_path, "contractions.json")
        self.load_resources()
        return

    def load_resources(self):
        self.stopwords = []
        self.synonym_dict = {}
        self.contractions = {}
        # stopwords
        with open(self.stopwords_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.stopwords = [x.rstrip() for x in f.readlines()]
            f.close()
        # special_stopwords
        with open(self.special_stopwords_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.stopwords.extend([x.rstrip() for x in f.readlines()])
            f.close()
        # special_characters
        with open(self.special_characters_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.stopwords.extend([x.rstrip() for x in f.readlines()])
            f.close()
        print("stopwords loaded.")
        # synonyms
        # POS <tab> Root_Word <tab> Synonym1 <tab> Synonym2 <tab> Synonym3
        # e.g. : verb    acknowledge    know    admit    understand
        with open(self.synonym_file, 'r', encoding='utf-8', errors='ignore') as f:
            synonym_list = [x.rstrip() for x in f.readlines()]
            for line in synonym_list:
                if len(line) < 4 or re.search("^#", line):
                    continue
                syn_words = line.lower().split("\t")  # root word
                pos = syn_words[0]                    # POS tag
                hwd = syn_words[1]                    # synonyms
                syn_words.pop(0)
                syn_words.pop(0)
                for s in syn_words:
                    self.synonym_dict.setdefault(s, {})[pos] = hwd
            f.close()
        print("synonyms_noun_verb loaded.")
        # contractions
        with open(self.contractions_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.contractions = dict(json.load(f))
            f.close()
        print("contractions loaded.")

    def get_pos_list(self, results):
        word_list, pos_ans, ne_list, start_end_list, lemma_list = [], [], [], [], []
        indices = results['sentences']
        for line in indices:
            tokens = line['tokens']
            for token in tokens:
                # 1. save tokens
                word_list.append(token['word'])
                # 2. save pos
                pos_syn = ""
                pos = token['pos'].lower()
                pos_ans.append(token['pos'])
                # 3. save lemmas
                lemma = token['lemma'].lower()
                # 3.1. filter lemmas
                if lemma in self.stopwords \
                    and lemma not in ['want', 'against', 'further', 'online', 'same', 'under','what', 'want', 'when', 'own'] \
                    or lemma in [":", "-lrb-", "-rrb-", "-lsb-", "-rsb-", "\\", '-pron-', '_', 'card num', '"'] \
                    or pos == ":" \
                    or pos == "." \
                    or re.search('^([\W]*)$]', lemma) \
                    or len(lemma) >= 30:
                    # print("filter:", lemma)
                    continue
                # 2.1. filter pos
                if re.search('^nn', pos):
                    pos_syn = 'noun'
                elif re.search('^v', pos):
                    pos_syn = 'verb'
                elif re.search('^adj', pos):
                    pos_syn = 'adj'
                # 3.2. filter lemmas again
                if lemma in list(self.synonym_dict.keys()) and pos_syn in list(self.synonym_dict[lemma].keys()):
                    lemma = self.synonym_dict[lemma][pos_syn]
                # save attributes
                lemma_list.append(lemma)
                ne_list.append(token['ner'])
                start_end_list.append(str(token['characterOffsetBegin']) + "_" + str(token['characterOffsetEnd']))
        return word_list, pos_ans, ne_list, start_end_list, lemma_list

    def basic_cleaning(self, input_sentences):
        cleaned_sentences = []
        for sent in input_sentences:
            sent = str(sent).strip()
            # FIX text
            sent = ftfy.fix_text(sent)
            # Normalize accented chars
            sent = unicodedata.normalize('NFKD', sent).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            # Removing <…> web scrape tags
            sent = re.sub(r"\<(.*?)\>", " ", sent)
            # Expanding contractions using contractions_file
            sent = re.sub(r"(\w+\'\w+)", lambda x: self.contractions.get(x.group().lower(), x.group().lower()), sent)
            # Removing web urls
            sent = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0–9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»""'']))''', " ", sent)
            # Removing date formats
            sent = re.sub(r"(\d{4}\-\d{2}\-\d{2}\s\d{2}\:\d{2}\:\d{2}\s\:)", " ", sent)
            # Removing extra whitespaces
            sent = re.sub(r"([\s\n\t\r]+)", " ", sent).strip()
            cleaned_sentences.append(sent)
        return cleaned_sentences

    def deep_cleaning(self, input_sentences):
        cleaned_sentences = []
        for sent in input_sentences:
            sent = str(sent).strip()
            sent = re.sub(r"\'s", " \'s", sent)
            sent = re.sub(r"\'ve", " \'ve", sent)
            sent = re.sub(r"n\'t", " n\'t", sent)
            sent = re.sub(r"\'re", " \'re", sent)
            sent = re.sub(r"\'d", " \'d", sent)
            sent = re.sub(r"\'ll", " \'ll", sent)
            sent = re.sub(r"[\$|\#\@\*\%]+\d+[\$|\#\@\*\%]+", " ", sent)
            sent = re.sub(r"(\d+\,\d+)", " ", sent)
            sent = re.sub(r"\;", " ", sent)
            sent = re.sub(r"\!", " ", sent)
            sent = re.sub(r"\(", " ", sent)
            sent = re.sub(r"\)", " ", sent)
            sent = re.sub(r"\?", " ", sent)
            sent = re.sub(r"\{", " ", sent)
            sent = re.sub(r"\}", " ", sent)
            sent = re.sub(r"\[", " ", sent)
            sent = re.sub(r"\]", " ", sent)
            sent = re.sub(r"\:", "", sent)
            sent = re.sub(r"\-", "", sent)
            sent = re.sub(r"\_", " ", sent)
            sent = re.sub(r"\s{2,}", " ", sent)
            sent = re.sub(r"[\/,\@,\#,\?,\\,\{,\},\(,\),\[,\],\$,\%,\^,\&,\*,\<,\>]", "", sent)
            sent = re.sub('\"', "", sent)
            sent = re.sub(r"'s\b", "", sent)
            sent = re.sub(r"\s+", " ", sent)
            # ---OPTIONAL---  keep only text & numbers
            sent = re.sub(r"\s+", " ", re.sub(r"[\\|\/|\||\{|\}|\[|\]\(|\)]+", " ", re.sub(r"[^A-z0-9]", " ", str(sent))))
            sent = sent.strip().lower()
            cleaned_sentences.append(sent)
        return cleaned_sentences

    def run_spacy(self, doc, operations='tokenize,ssplit,pos,lemma,ner'):
        """
        Spacy nlp pipeline to generate spacy text features such as pos, tokens, ner, dependency.
        Accepts doc=nlp(text)
        """
        # spacy doc
        doc_json = doc.to_json()  # Includes all operations given by spacy pipeline

        # Get text
        text = doc_json['text']

        # ---------------------------------------- OPERATIONS  -----------------------------------------
        # 1. Extract Entity List
        entity_list = doc_json["ents"]

        # 2. Create token lib
        token_lib = {token["id"]: token for token in doc_json["tokens"]}

        # init output json
        output_json = {}
        output_json["sentences"] = []

        # Perform spacy operations on each sent in text
        for i, sentence in enumerate(doc_json["sents"]):

            # init parsers
            parse = ""
            basicDependencies = []
            enhancedDependencies = []
            enhancedPlusPlusDependencies = []

            # init output json
            out_sentence = {"index": i, "line": 1, "tokens": []}
            output_json["sentences"].append(out_sentence)

            # 3. Split sentences by indices(i), add labels (pos, ner, dep, etc.)
            for token in doc_json["tokens"]:

                if sentence["start"] <= token["start"] and token["end"] <= sentence["end"]:
                    # >>> Extract Entity label
                    ner = "O"
                    for entity in entity_list:
                        if entity["start"] <= token["start"] and token["end"] <= entity["end"]:
                            ner = entity["label"]

                    # >>> Extract dependency info
                    dep = token["dep"]
                    governor = 0 if token["head"] == token["id"] else (token["head"] + 1)  # CoreNLP index = pipeline index +1
                    governorGloss = "ROOT" if token["head"] == token["id"] else text[token_lib[token["head"]]["start"]:
                                                                                     token_lib[token["head"]]["end"]]
                    dependent = token["id"] + 1
                    dependentGloss = text[token["start"]:token["end"]]

                    # >>> Extract lemma
                    lemma = doc[token["id"]].lemma_

                    # 4. Add dependencies
                    basicDependencies.append({"dep": dep,
                                              "governor": governor,
                                              "governorGloss": governorGloss,
                                              "dependent": dependent,
                                              "dependentGloss": dependentGloss})

                    # 5. Add tokens
                    out_token = {"index": token["id"] + 1,
                                 "word": dependentGloss,
                                 "originalText": dependentGloss,
                                 "characterOffsetBegin": token["start"],
                                 "characterOffsetEnd": token["end"]}

                    # 6. Add lemmas
                    if "lemma" in operations:
                        out_token["lemma"] = lemma

                    # 7. Add POS tagging
                    if "pos" in operations:
                        out_token["pos"] = token["tag"]

                    # 8. Add NER
                    if "ner" in operations:
                        out_token["ner"] = ner

                    # Update output json
                    out_sentence["tokens"].append(out_token)

            # 9. Add dependencies operation
            if "parse" in operations:
                out_sentence["parse"] = parse
                out_sentence["basicDependencies"] = basicDependencies
                out_sentence["enhancedDependencies"] = out_sentence["basicDependencies"]
                out_sentence["enhancedPlusPlusDependencies"] = out_sentence["basicDependencies"]
        # ---------------------------------------- OPERATIONS  -----------------------------------------
        return output_json

    # main
    def clean(self, input_sentences, operations=["basic","deep","spacy"]):
        """
        Main module to execute pipeline. Accepts list of strings, and desired operations.
        """
        sentences = input_sentences

        # run basic cleaning
        if "basic" in list(map(str.lower, operations)): sentences = self.basic_cleaning(sentences)

        # run deep cleaning
        if "deep" in list(map(str.lower, operations)): sentences = self.deep_cleaning(sentences)

        # run spacy pipeline
        if "spacy" in list(map(str.lower, operations)):
            batch_size = min(int(np.ceil(len(input_sentences)/100)), 500)
            sentences = [self.run_spacy(doc) for doc in nlp.pipe(sentences, batch_size=batch_size, n_threads=-1)]
            # save spacy results
            # word_list, pos_list, ner_list, start_end_list, lemma_list
            spacy_sentences = []
            for sent in sentences:
                word_list, pos_ans, ne_list, start_end_list, lemma_list = self.get_pos_list(sent)
                spacy_sentences.append({"word_list": word_list,
                                        "pos_list": pos_ans,
                                        "ner_list": ne_list,
                                        "start_end_list": start_end_list,
                                        "lemma_list": lemma_list})
            sentences = spacy_sentences

        return sentences

#### Run pre-process

In [0]:
# RESOURCES DIRECTORY

# chatwords.txt
"""
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

## contractions.json
"""
{"it's": "it is", "o'clock": "of the clock", "I'd've": "I would have", "how's": "how is", "you'll": "you will", "could've": "could have", "that'd've": "that would have", "I'll've": "I will have", "she's": "she is", "there'd": "there would", "you've": "you have", "why've": "why have", "should've": "should have", "he's": "he is", "'cause": "because", "y'all're": "you all are", "i'd": "i would", "it'll've": "it will have", "they'd've": "they would have", "when's": "when is", "they'll": "they will", "we'd've": "we would have", "where've": "where have", "will've": "will have", "it'd": "it would", "they'll've": "they will have", "couldn't": "could not", "hasn't": "has not", "who'll've": "who will have", "shan't": "shall not", "we're": "we are", "y'all'd": "you all would", "hadn't": "had not", "doesn't": "does not", "aren't": "are not", "let's": "let us", "that'd": "that would", "you'd've": "you would have", "y'all": "you all", "won't": "will not", "isn't": "is not", "can't": "cannot", "ma'am": "madam", "wouldn't've": "would not have", "who've": "who have", "what'll": "what will", "don't": "do not", "y'all'd've": "you all would have", "haven't": "have not", "how'd'y": "how do you", "what's": "what is", "might've": "might have", "we've": "we have", "would've": "would have", "mightn't've": "might not have", "where'd": "where did", "mightn't": "might not", "she'd": "she would", "how'd": "how did", "won't've": "will not have", "y'all've": "you all have", "oughtn't": "ought not", "here's": "here is", "I'd": "I would", "mayn't": "may not", "why's": "why is", "who'll": "who will", "shouldn't": "should not", "I'm": "I am", "they're": "they are", "what're": "what are", "when've": "when have", "she'll": "she will", "mustn't": "must not", "what've": "what have", "I'll": "I will", "you're": "you are", "he'll": "he will", "there's": "there is", "must've": "must have", "weren't": "were not", "it'll": "it will", "who's": "who is", "to've": "to have", "needn't've": "need not have", "they'd": "they would", "that's": "that is", "oughtn't've": "ought not have", "they've": "they have", "i'll": "i will", "wouldn't": "would not", "you'd": "you would", "i've": "i have", "shan't've": "shall not have", "shouldn't've": "should not have", "i'd've": "i would have", "we'd": "we would", "ain't": "is not", "mustn't've": "must not have", "this's": "this is", "it'd've": "it would have", "what'll've": "what will have", "we'll've": "we will have", "she'd've": "she would have", "where's": "where is", "he'd": "he would", "so've": "so have", "so's": "so as", "you'll've": "you will have", "sha'n't": "shall not", "she'll've": "she will have", "how'll": "how will", "i'll've": "i will have", "wasn't": "was not", "needn't": "need not", "I've": "I have", "we'll": "we will", "there'd've": "there would have", "didn't": "did not", "i'm": "i am", "re ":"re"}
"""

# emojis.txt
"""
u"\U0001F600-\U0001F64F" 
u"\U0001F300-\U0001F5FF"  
u"\U0001F680-\U0001F6FF"  
u"\U0001F1E0-\U0001F1FF"  
u"\U00002702-\U000027B0"
u"\U000024C2-\U0001F251"
"""

# emoticons.json
"""
{
    ":\u2011\\)": "Happy face or smiley",
    ":\\)": "Happy face or smiley",
    ":-\\]": "Happy face or smiley",
    ":\\]": "Happy face or smiley",
    ":-3": "Happy face smiley",
    ":3": "Happy face smiley",
    ":->": "Happy face smiley",
    ":>": "Happy face smiley",
    "8-\\)": "Happy face smiley",
    ":o\\)": "Happy face smiley",
    ":-\\}": "Happy face smiley",
    ":\\}": "Happy face smiley",
    ":-\\)": "Happy face smiley",
    ":c\\)": "Happy face smiley",
    ":\\^\\)": "Happy face smiley",
    "=\\]": "Happy face smiley",
    "=\\)": "Happy face smiley",
    ":\u2011D": "Laughing, big grin or laugh with glasses",
    ":D": "Laughing, big grin or laugh with glasses",
    "8\u2011D": "Laughing, big grin or laugh with glasses",
    "8D": "Laughing, big grin or laugh with glasses",
    "X\u2011D": "Laughing, big grin or laugh with glasses",
    "XD": "Laughing, big grin or laugh with glasses",
    "=D": "Laughing, big grin or laugh with glasses",
    "=3": "Laughing, big grin or laugh with glasses",
    "B\\^D": "Laughing, big grin or laugh with glasses",
    ":-\\)\\)": "Very happy",
    ":\u2011\\(": "Frown, sad, andry or pouting",
    ":-\\(": "Frown, sad, andry or pouting",
    ":\\(": "Frown, sad, andry or pouting",
    ":\u2011c": "Frown, sad, andry or pouting",
    ":c": "Frown, sad, andry or pouting",
    ":\u2011<": "Frown, sad, andry or pouting",
    ":<": "Frown, sad, andry or pouting",
    ":\u2011\\[": "Frown, sad, andry or pouting",
    ":\\[": "Frown, sad, andry or pouting",
    ":-\\|\\|": "Frown, sad, andry or pouting",
    ">:\\[": "Frown, sad, andry or pouting",
    ":\\{": "Frown, sad, andry or pouting",
    ":@": "Frown, sad, andry or pouting",
    ">:\\(": "Frown, sad, andry or pouting",
    ":'\u2011\\(": "Crying",
    ":'\\(": "Crying",
    ":'\u2011\\)": "Tears of happiness",
    ":'\\)": "Tears of happiness",
    "D\u2011':": "Horror",
    "D:<": "Disgust",
    "D:": "Sadness",
    "D8": "Great dismay",
    "D;": "Great dismay",
    "D=": "Great dismay",
    "DX": "Great dismay",
    ":\u2011O": "Surprise",
    ":O": "Surprise",
    ":\u2011o": "Surprise",
    ":o": "Surprise",
    ":-0": "Shock",
    "8\u20110": "Yawn",
    ">:O": "Yawn",
    ":-\\*": "Kiss",
    ":\\*": "Kiss",
    ":X": "Kiss",
    ";\u2011\\)": "Wink or smirk",
    ";\\)": "Wink or smirk",
    "\\*-\\)": "Wink or smirk",
    "\\*\\)": "Wink or smirk",
    ";\u2011\\]": "Wink or smirk",
    ";\\]": "Wink or smirk",
    ";\\^\\)": "Wink or smirk",
    ":\u2011,": "Wink or smirk",
    ";D": "Wink or smirk",
    ":\u2011P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "X\u2011P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "XP": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":\u2011\u00de": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":\u00de": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":b": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "d:": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "=p": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ">:P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":\u2011/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":-[.]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ">:[(\\\\)]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ">:/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":[(\\\\)]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    "=/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    "=[(\\\\)]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":L": "Skeptical, annoyed, undecided, uneasy or hesitant",
    "=L": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":S": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":\u2011\\|": "Straight face",
    ":\\|": "Straight face",
    ":$": "Embarrassed or blushing",
    ":\u2011x": "Sealed lips or wearing braces or tongue-tied",
    ":x": "Sealed lips or wearing braces or tongue-tied",
    ":\u2011#": "Sealed lips or wearing braces or tongue-tied",
    ":#": "Sealed lips or wearing braces or tongue-tied",
    ":\u2011&": "Sealed lips or wearing braces or tongue-tied",
    ":&": "Sealed lips or wearing braces or tongue-tied",
    "O:\u2011\\)": "Angel, saint or innocent",
    "O:\\)": "Angel, saint or innocent",
    "0:\u20113": "Angel, saint or innocent",
    "0:3": "Angel, saint or innocent",
    "0:\u2011\\)": "Angel, saint or innocent",
    "0:\\)": "Angel, saint or innocent",
    ":\u2011b": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "0;\\^\\)": "Angel, saint or innocent",
    ">:\u2011\\)": "Evil or devilish",
    ">:\\)": "Evil or devilish",
    "\\}:\u2011\\)": "Evil or devilish",
    "\\}:\\)": "Evil or devilish",
    "3:\u2011\\)": "Evil or devilish",
    "3:\\)": "Evil or devilish",
    ">;\\)": "Evil or devilish",
    "\\|;\u2011\\)": "Cool",
    "\\|\u2011O": "Bored",
    ":\u2011J": "Tongue-in-cheek",
    "#\u2011\\)": "Party all night",
    "%\u2011\\)": "Drunk or confused",
    "%\\)": "Drunk or confused",
    ":-###..": "Being sick",
    ":###..": "Being sick",
    "<:\u2011\\|": "Dump",
    "\\(>_<\\)": "Troubled",
    "\\(>_<\\)>": "Troubled",
    "\\(';'\\)": "Baby",
    "\\(\\^\\^>``": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    "\\(\\^_\\^;\\)": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    "\\(-_-;\\)": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    "\\(~_~;\\) \\(\u30fb\\.\u30fb;\\)": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    "\\(-_-\\)zzz": "Sleeping",
    "\\(\\^_-\\)": "Wink",
    "\\(\\(\\+_\\+\\)\\)": "Confused",
    "\\(\\+o\\+\\)": "Confused",
    "\\(o\\|o\\)": "Ultraman",
    "\\^_\\^": "Joyful",
    "\\(\\^_\\^\\)/": "Joyful",
    "\\(\\^O\\^\\)\uff0f": "Joyful",
    "\\(\\^o\\^\\)\uff0f": "Joyful",
    "\\(__\\)": "Kowtow as a sign of respect, or dogeza for apology",
    "_\\(\\._\\.\\)_": "Kowtow as a sign of respect, or dogeza for apology",
    "<\\(_ _\\)>": "Kowtow as a sign of respect, or dogeza for apology",
    "<m\\(__\\)m>": "Kowtow as a sign of respect, or dogeza for apology",
    "m\\(__\\)m": "Kowtow as a sign of respect, or dogeza for apology",
    "m\\(_ _\\)m": "Kowtow as a sign of respect, or dogeza for apology",
    "\\('_'\\)": "Sad or Crying",
    "\\(/_;\\)": "Sad or Crying",
    "\\(T_T\\) \\(;_;\\)": "Sad or Crying",
    "\\(;_;": "Sad of Crying",
    "\\(;_:\\)": "Sad or Crying",
    "\\(;O;\\)": "Sad or Crying",
    "\\(:_;\\)": "Sad or Crying",
    "\\(ToT\\)": "Sad or Crying",
    ";_;": "Sad or Crying",
    ";-;": "Sad or Crying",
    ";n;": "Sad or Crying",
    ";;": "Sad or Crying",
    "Q\\.Q": "Sad or Crying",
    "T\\.T": "Sad or Crying",
    "QQ": "Sad or Crying",
    "Q_Q": "Sad or Crying",
    "\\(-\\.-\\)": "Shame",
    "\\(-_-\\)": "Shame",
    "\\(\u4e00\u4e00\\)": "Shame",
    "\\(\uff1b\u4e00_\u4e00\\)": "Shame",
    "\\(=_=\\)": "Tired",
    "\\(=\\^\\\u00b7\\^=\\)": "cat",
    "\\(=\\^\\\u00b7\\\u00b7\\^=\\)": "cat",
    "=_\\^=\t": "cat",
    "\\(\\.\\.\\)": "Looking down",
    "\\(\\._\\.\\)": "Looking down",
    "\\^m\\^": "Giggling with hand covering mouth",
    "\\(\\\u30fb\\\u30fb?": "Confusion",
    "\\(?_?\\)": "Confusion",
    ">\\^_\\^<": "Normal Laugh",
    "<\\^!\\^>": "Normal Laugh",
    "\\^/\\^": "Normal Laugh",
    "\\\uff08\\*\\^_\\^\\*\uff09": "Normal Laugh",
    "\\(\\^<\\^\\) \\(\\^\\.\\^\\)": "Normal Laugh",
    "\\(^\\^\\)": "Normal Laugh",
    "\\(\\^\\.\\^\\)": "Normal Laugh",
    "\\(\\^_\\^\\.\\)": "Normal Laugh",
    "\\(\\^_\\^\\)": "Normal Laugh",
    "\\(\\^\\^\\)": "Normal Laugh",
    "\\(\\^J\\^\\)": "Normal Laugh",
    "\\(\\*\\^\\.\\^\\*\\)": "Normal Laugh",
    "\\(\\^\u2014\\^\\\uff09": "Normal Laugh",
    "\\(#\\^\\.\\^#\\)": "Normal Laugh",
    "\\\uff08\\^\u2014\\^\\\uff09": "Waving",
    "\\(;_;\\)/~~~": "Waving",
    "\\(\\^\\.\\^\\)/~~~": "Waving",
    "\\(-_-\\)/~~~ \\($\\\u00b7\\\u00b7\\)/~~~": "Waving",
    "\\(T_T\\)/~~~": "Waving",
    "\\(ToT\\)/~~~": "Waving",
    "\\(\\*\\^0\\^\\*\\)": "Excited",
    "\\(\\*_\\*\\)": "Amazed",
    "\\(\\*_\\*;": "Amazed",
    "\\(\\+_\\+\\) \\(@_@\\)": "Amazed",
    "\\(\\*\\^\\^\\)v": "Laughing,Cheerful",
    "\\(\\^_\\^\\)v": "Laughing,Cheerful",
    "\\(\\(d[-_-]b\\)\\)": "Headphones,Listening to music",
    "\\(-\"-\\)": "Worried",
    "\\(\u30fc\u30fc;\\)": "Worried",
    "\\(\\^0_0\\^\\)": "Eyeglasses",
    "\\(\\\uff3e\uff56\\\uff3e\\)": "Happy",
    "\\(\\\uff3e\uff55\\\uff3e\\)": "Happy",
    "\\(\\^\\)o\\(\\^\\)": "Happy",
    "\\(\\^O\\^\\)": "Happy",
    "\\(\\^o\\^\\)": "Happy",
    "\\)\\^o\\^\\(": "Happy",
    ":O o_O": "Surprised",
    "o_0": "Surprised",
    "o\\.O": "Surpised",
    "\\(o\\.o\\)": "Surprised",
    "oO": "Surprised",
    "\\(\\*\uffe3m\uffe3\\)": "Dissatisfied",
    "\\(\u2018A`\\)": "Snubbed or Deflated"
}
"""

# greeting_words.txt
"""
"awesome to see you",
"dear",
"dear sir",
"dear madam",
"dear ma'am",
"dear friend",
"dear sister",
"dear brother",
"dear sire",
"dear sirs",
"dear sir/madam",
"dear maam",
"dear maams",
"feels warm to see you",
"feels good to see you",
"feels pretty good to see you",
"feels nice to see you",
"feels awesome to see you",
"feels great to see you",
"feels wonderful to see you",
"gentlemen and ladies",
"gmorning",
"good morning",
"goodmorning",
"morning",
"great day",
"good day",
"g'day",
"g' day",
"good afternoon",
"g'afternoon",
"g'noon",
"g' noon",
"good evening",
"g'evening",
"g' evening",
"g'eve",
"good greeting",
"good greetings",
"greetings",
"greeting",
"good to see you",
"pretty good to see you",
"great to see you",
"hi",
"hello",
"hey",
"helloo",
"hellooo",
"howdy",
"heyyou",
"heyy",
"hii",
"hie",
"hola",
"hii",
"heyi",
"heyy",
"hiii",
"helloo",
"heya",
"hey there",
"hi there",
"hello there",
"hi team",
"hello team",
"hey team",
"how are you",
"how're you",
"how are you doing",
"how ya doin'",
"how ya doin",
"how is everything",
"how is everything going",
"how's everything going",
"how is you",
"how's you",
"how is it going",
"how's it going",
"how's it goin'",
"how's it goin",
"how's life treating you",
"how is life treating you",
"how is life been treating you",
"how's life been treating you",
"how are thing",
"how're thing",
"how're things",
"how are things",
"how're things with you",
"how are things with you",
"how you'd been",
"how you had been",
"how you been",
"how have you been",
"how've you been",
"its good seeing you",
"its nice seeing you",
"its great seeing you",
"its awesome seeing you",
"its wonderful seeing you",
"ladies and gentlemen",
"ladies & gentlemen",
"ladies and gents",
"ladies & gents",
"mornin",
"moriningg",
"morining",
"nice to see you",
"obliged to see you",
"respected",
"respected teacher",
"respected concerned",
"respected sir",
"respected madam",
"respected ma'am",
"respected sire",
"respected sirs",
"respected sir/madam",
"respected maam",
"respected maams",
"very nice to see you",
"wonderful to see you",
"what is up",
"what's up",
"what's upp",
"what is upp",
"what is cracking",
"what's cracking",
"what is good",
"what's good",
"what is happening",
"what's happening",
"what is new",
"what's new",
"what is neww",
"""

# signature_words.txt
"""
"all best",
"all the best",
"all the bests",
"all my best",
"all my regards",
"all my regard",
"all my regds",
"all my rgds",
"best wishes",
"best regards",
"best regard",
"best regds",
"best rgds",
"bGIF",
"ciao",
"cordially",
"cheers",
"kind regards",
"kind regard",
"kind regds",
"kind rgds",
"kindest regards",
"kindest regard",
"kindest regds",
"kindest rgds",
"many cheers",
"many thanks",
"many many thanks",
"my best",
"my regards",
"my regard",
"my regds",
"my rgds",
"sent from my android",
"sent from my iphone",
"sent from my phone",
"sincerely",
"regards",
"regard",
"regds",
"rgds",
"thanks",
"thanks and regards",
"thanks and regard",
"thanks and regrds",
"thanks and regrd",
"thanks and rgards",
"thanks and rgard",
"thanks and regds",
"thanks and regd",
"thanks and rgds",
"thanks and rgd",
"thanks & regards",
"thanks & regard",
"thanks & regrds",
"thanks & regrd",
"thanks & rgards",
"thanks & rgard",
"thanks & regds",
"thanks & regd",
"thanks & rgds",
"thanks & rgd",
"thankx and regards",
"thankx and regard",
"thx and regards",
"thankx & regards",
"thankx & regard",
"thx & regards",
"thank and regards",
"thank and regard",
"thank you and regards",
"thank you and regard",
"thankyou and regards",
"thankyou and regard",
"thank you & regards",
"thank you & regard",
"thankyou & regards",
"thankyou & regard",
"thank & regards",
"thank & regard",
"thanking you and regards",
"thanking you and regard",
"thankingyou and regards",
"thankingyou and regard",
"thanks so much",
"thanks you",
"thank you",
"thankyou",
"thxxx",
"thankingu",
"thankyou so much",
"thanking you",
"thankingyou",
"thank you so much",
"take care",
"talk soon",
"will talk soon",
"will talk very soon",
"will talk sooner",
"warmest",
"warm regards", 
"warm regds",
"warm rgds",
"warm regard", 
"warmest regards",
"warmest regds",
"warmest rgds",
"warmest regard",
"warmly",
"with many cheers",
"with many thanks",
"with warmth",
"with warmest",
"with warm regards",
"with warm regds",
"with warm rgds",
"with warm regard",
"with regards",
"with regard",
"with regds",
"with rgds",
"your cordially",
"yours cordially",
"your truly",
"yours truly",
"your sincerely",
"yours sincerely",
"your loving",
"yours loving",
"your lovingly",
"yours lovingly",
"your honestly",
"yours honestly"
"""

# special_characters.txt
"""
,
.
"
:
)
(
-
!
?
|
;
'
$
&
/
[
]
>
%
=
#
*
+
\\
•
~
@
£
·
_
{
}
©
^
®
`
<
→
°
€
™
›
♥
←
×
§
″
′
Â
█
½
à
…
"
""
★
–
●
â
►
−
¢
²
¬
░
¶
↑
±
¿
▾
═
¦
║
―
¥
▓
 - 
--
‹
─
▒
：
¼
⊕
▼
▪
†
■
'
▀
¨
▄
♫
☆
é
¯
♦
¤
▲
è
¸
¾
Ã
⋅
'
∞
∙
）
↓
、
│
（
»
，
♪
╩
╚
³
・
╦
╣
╔
╗
▬
❤
ï
Ø
¹
≤
‡
√
"""
    
# special_stopwords.txt
"""
https
http
india
usa
com
team
please
pls
let
know
respond
name
jan
feb
mar
apr
may
jun
jul
aug
sep
oct
nov
dec
asap
january
february
march
april
june
july
august
september
october
november
december
dear
kindly
regards
screenshot
sunday
monday
tuesday
wednesday
thursday
friday
saturday
pfa
bangalore
mumbai
pune
thanks
thank
thanking
thanks
thnx
thankyou
thanksyou
takecare
welcome
grateful
yep
yeah
nope
noo
"""
    
# stopwords.txt
"""
``
''
.
,
'
"
/
?
-
:
#
-lsb-
-rsb-
a
at
about
above
after
again
against
all
am
an
and
any
are
as
be
because
been
before
being
below
between
both
but
did
does
do
doing
don
down
during
each
few
for
from
further
had
has
have
having
he
here
hers
herself
him
himself
i
if
into
is
it
its
itself
just
me
more
most
myself
nor
noo
no
nope
nada
now
off
once
online
only
or
other
ours
ourselves
out
over
own
please
's
s
same
should
so
some
such
t
than
that
the
theirs
them
themselves
then
there
these
this
those
through
too
under
until
up
very
want
was
were
while
will
with
w8
w88
you
yo
yay
yoo
yeda
yuu
yooo
&
my
of
on
she
to
we
in
they
"""
    
# synonyms_noun_verb.txt
"""
verb	acknowledge	know	admit	understand
verb	activate	trigger
verb	aggregate	accumulate	collect	gather
verb	alert	notify	inform
verb	allocate	allot	designate
verb	allow	permit
verb	answer	reply	respond
verb	approve	accept	validate
verb	ask	inquire
verb	attempt	try	endeavor
verb	calculate	compute
verb	cancel	terminate	revoke	void	waive
verb	check	examine	inspect
verb	circumvent	avoid	bypass	sidestep
verb	close	shut	shut down	turn off
verb	compare	contrast	collate	differenciate
verb	comply	agree	confirm	acknowledge	satisfy	obey
verb	concentrate	focus
verb	connect	join	link
verb	consist	comprise	include	contain
verb	consolidate	unify
verb	contact	associate
verb	continue	maintain
verb	decease	die	expire
verb	decide	determine
verb	deduct	subtract	withhold
verb	defend	protect	guard
verb	define	describe
verb	delay	lag	postpone	hold-up
verb	delete	remove	erase	eliminate
verb	delink	disconnect	de-link
verb	describe	explain
verb	disclose	reveal
verb	donate	contribute
verb	duplicate	copy	replicate
verb	edit	revise	correct	emend	change	modify	customize	amend	alter	revise	fix	update
verb	energize	invigorate	stimulate	strenghthen
verb	enlarge	expand	magnify
verb	enroll	register
verb	entail	encompass	involve	comprise
verb	estimate	appraisal	assessment	measure
verb	excess	surplus
verb	exclude	eliminate	omit
verb	execute	carry out
verb	fill	fill out
verb	find	discover	find out	recover
verb	fix	correct	resolve
verb	follow	come after
verb	forbid	prohibit	ban
verb	generate	yield	bring forth
verb	get	acquire	obtain
verb	gray	gray out	grey	grey out
verb	handle	treat	deal	manage
verb	happen	occur	take place
verb	help	aid	assist	support
verb	hide	conceal
verb	hurry	rush	hasten	accelerate	expedite
verb	import	bring into
verb	invest	fund
verb	leave	exit	exit out
verb	lend	loan
verb	liquidate	dissolve	convert to cash	sell off
verb	log in	access
verb	mark	label	tag
verb	mean	imply	denote	signify
verb	meet	encounter
verb	merge	unify	unite	blend	fuse	combine
verb	print	print out
verb	prove	authenticate	verify	validate
verb	provide	supply	cater	offer
verb	pull	pull up
verb	raise	lift	elevate	bring up
verb	remit	send	send out	resend	submit	resubmit
verb	return	come back	refund	give back
verb	review	reexamine
verb	search	look for	seek	browse
verb	see	view	watch
verb	separate	divide	split	split up	break up
verb	show	show up	display	reveal	unhide
verb	speak	talk
verb	start	begin	commence	launch
verb	stop	cease
verb	store	save
verb	think	consider
verb	transfer	transfer out
verb	un-enroll	unenroll	uneroll
verb	unhide	show
verb	use	utilize	apply
verb	want	would like
# noun synonyms
noun	ability	capability
noun	account	acct	accnt	accts	accnts	acccount	accont	acc	acct	accountss	accunts	accts	accountss	accnts	accoun	accnts	
noun	accuracy	precision	correctness
noun	addendum	appendix
noun	address	location	addrees	addres	addtress	adresss
noun	adjustment	modification	amendment
noun	admin	administer	administrator
noun	advice	suggestion
noun	affidavit	oath	statement
noun	aggregation	addition
noun	agreement	consensus
noun	allocation	allotment	portion
noun	alternative	substitute	back-up
noun	analysis	analytics
noun	appointment	meeting
noun	approval	consent	permission	
noun	arbitration	adjudication	judgment
noun	article	report	piece
noun	asset	property
noun	assistance	help	support
noun	attachment	accessory
noun	average	mediocre	
noun	award	prize	trophy
noun	backup	substitute	alternate
noun	bank	financial institution	finance company
noun	bankruptcy	insolvency
noun	baseline	basis
noun	batch	group	lot
noun	benchmark	standard
noun	beneficial	benefit	advantage
noun	beneficiary	inheritor	heir
noun	board	committee
noun	bond	fixed-income securities	corporate bonds	government bonds
noun	branch	division	department
noun	broker	discount brokers	full service brokers
noun	brokerage	brokerage company
noun	building	construction	
noun	buyer	purchaser	shopper	customer
noun	calculation	computation
noun	cancellation	anullment
noun	capacity	quantity
noun	cashier	clerk	teller	treasurer
noun	category	class
noun	character	personality	disposition
noun	charity	goodwill
noun	chart	graph
noun	citizenship	nationality
noun	claim	demand
noun	client	user	customer
noun	closure	termination	shutdown
noun	coin	money	dollar	currency	cash
noun	college	university
noun	comment	remark
noun	commodity	merchandise	goods	product
noun	competitor	challenger	opponent
noun	compliance	conformity
noun	component	constituent
noun	consequence	result	outcome
noun	conservator	custodian	guardian
noun	context	situation	circumstances
noun	continuation	maintenance
noun	contract	arrangement
noun	conversation	talk
noun	conversion	alteration
noun	corporate	company
noun	correct	accurate	precise
noun	credit	loan	advance
noun	custodial	protective
noun	custodian	protector	guardian
noun	cutoff	interruption	pause
noun	cycle	course	period
noun	dashboard	home page
noun	death	decease	demise	casualty
noun	debit	payment made
noun	debt	liability	amount owed
noun	decent	good	respectable
noun	decision	judgement	choice
noun	default	nonpayment
noun	delivery	shipment	consignment	distribute
noun	edelivery	edelivey	edleivery	edlivery	eedelivery
noun	verb	differ	diverge	diversify	reverse
noun	difference	distinction	divergence	
noun	disbursement	expenditure	spending
noun	disclosure	announcement	revelation
noun	discount	concession
noun	discrepancy	conflict	disagreement
noun	distribution	dispersion
noun	dividend	gain	premium	return
noun	divorce	breakup	separation
noun	document	file
noun	draft	plan	blueprint
noun	durable	reliable	lasting	dependable	stable
noun	effective	productive	efficient
adj	electronic	electronically
noun	eligibility	qualification
noun	employee	worker
noun	work	works
noun	trade	trading
noun	enforcement	imposition
noun	sub-user	subuser	subusername
noun	enough	adequate	sufficient
noun	registration	enrollment
adj	entire	complete	whole
noun	phone	iphone
noun	mercedes	mercedes-benz
noun	entitlement	privilege	authorization
noun	equity	share	stock
noun	error	mistake	wrong	inaccuracy
noun	escalation	increase	growth	rise
noun	escheatment	transfer of financial assets
noun	escrow	bond	deed
noun	esign	electronic signature
noun	evaluation	assessment	appraisal	judgement
noun	example	instance
noun	exception	barring	omission
noun	exist	live	occur
noun	experience	maturity	knowledge
noun	expiration	termination
noun	explanation	clarification
noun	extension	continuation
noun	extensive	thorough
adj	external	extrinsic	outside
noun	facility	convenience	ease
noun	family	kin	beneficiary
noun	feature	characteristic
noun	fee	charge
noun	financial	fiscal	economic
noun	flag	marker	indicator
noun	fluctuate	oscillate	waver
noun	force	effort
noun	foreign	alien	overseas
noun	forfeiture	loss
noun	instructore	mentor	tutor
adj	internal	intrinsic	inside
noun	mobile	cell
noun	mso	msonline
noun	password	pin	key	passcode	passsword	passward	passwod	passwrd
noun	statement	stmnt	stmt
noun	supervisor	director

noun	3D Advisor Desktop	3D
noun	Active Assets Account	AAA
noun	Morgan Stanley Active Assets California Tax Free Trust	AACT
noun	Morgan Stanley Active Assets California Tax-Free Trust	AACT
noun	Morgan Stanley Active Assets Tax Free Trust	AAIF
noun	Morgan Stanley Active Assets Tax-Free Trust	AAIF
noun	Morgan Stanley Active Assets Institutional Government Securities Trust	AAIG
noun	New York Municipal Money Market Trust	AANY
noun	Morgan Stanley Active Assets Tax Free Trust	AATF
noun	Morgan Stanley Active Assets Tax-Free Trust	AATF
noun	American Bankers Association	ABA
noun	American Banking Association	ABA
noun	Automated Customer Account Transfer	ACATS
noun	Account Cross Reference	ACCR
noun	Automated Clearing House	ACH
noun	Automatic Cash Management	ACM
noun	Automated Confirmation Transactions	ACT
noun	Adjustment	ADJ
noun	Automatic Data Processing	ADP
noun	American Depository Receipt	ADR
noun	Alternative Flexible Grid	AFG
noun	Automatic Funds Transfer	AFT
noun	Alternative Investment	AI
noun	Alternative Investments Advisory	AIA
noun	American Jobs Creation Act	AJCA
noun	Account Link Group	ALG
noun	American Stock Exchange	AMEX
noun	Admin Wire System	AMIN
noun	Anti-Money Laundering	AML
noun	Account Management Portal	AMP
noun	Alternative Minimum Tax	AMT
noun	Advice of Correction or Cancellation	AOCC
noun	Advice of Correction/Cancellation	AOCC
noun	Address of Record	AOR
noun	Asia Pacific regions	APJ
noun	Asia Pacific regions	APJJ
noun	Army Post Office	APO
noun	Actuarial Present Value	APV
noun	Advanced Resources Center	ARC
noun	Auction Rate Preferred Securities	ARPS
noun	Account Redistribution System	ARS
noun	Account Surveillance and Fraud	AS&F
noun	Automated Stock Access Program	ASAP
noun	Automatic Segregation	ASG
noun	Application Service Provider	ASP
noun	Automated Trade Adjustment	ATA
noun	Account Type Change	ATC
noun	Automatic Tracking Report Distribution	ATRD
noun	Advisory Trade Violations	ATV
noun	Australian Dollar	AUD
noun	Assets Under Management	AUM
noun	Automated Required Minimum Distribution	Auto-RMD
noun	Auxiliary Cross Reference Criteria	AUX
noun	Auxiliary Cross Reference Criteria	AuxOnUs
noun	Automated Workflow Director	AWD
noun	Buy and Hold	B&H
noun	Benefit Access	BA
noun	Benefit Access Funds Transfers	BAFT
noun	Bond Anticipation Note	BAN
noun	Benefit Access Redesign	BAR
noun	Branch Based Manual Stock Plan	BBMSP
noun	Business Continuity Planning	BCO
noun	Banking and Cash Services	BCS
noun	Business Development Allowance	BDA
noun	Bank Deposit Program	BDP
noun	Bank Deposit Program Sweep	BDPS
noun	Broker Dealer Questionnaire	BDQ
noun	Broker-Dealer Questionnaire	BDQ
noun	Branch Deposit Scanning	BDS
noun	Boston Financial Dealer Services	BFDS
noun	Bank Identifier Code	BIC
noun	Best Interest Contract	BIC
noun	Broker Identification Number	BIN
noun	Branch Inquiry Tracking System	BITS
noun	Bookkeeping and Stock Record	BKSR
noun	Banking and Lending Support	BLS
noun	Bond Market Association	BMA
noun	Branch Manager Dashboard	BMD
noun	Branch Office Manager	BOM
noun	Bank of New York	BONY
noun	Branch Office Support System	BOSS
noun	Branch Office Terminal Application System	BOTA
noun	Business Process Management	BPM
noun	Basis Points	BPS
noun	Brokerage Processing System	BPS
noun		BPS Data Base System Advantage
noun	Business Remote Deposit	BRD
noun	Business Requirement Document	BRD
noun	Boston Stock Exchange	BSE
noun	Brokerage Service Group	BSG
noun	Business Service Manager	BSM
noun	Broker Terminal System	BTS
noun	Broker Transaction System	BTS
noun	Broker Transaction System 010	BTS
noun	Broker Transaction System - 232	BTS 1
noun	Broker Transaction System 232	BTS 1
noun	Business Unit	BU
noun	Business Unit Indicator	BUI
noun	Corporate Actions	CA
noun	Client Advisory Center	CAC
noun	Canadian Dollar	CAD
noun	Custom Account Groups	CAG
noun	Cash Account Management System	CAMS
noun	Corporate Actions Notifications and Response	CANR
noun	Capital Accumulation Plan	CAP
noun	Class Action Query	CAQ
noun	Corporate Benefit Access	CBA
noun	Cost Benefit Analysis	CBA
noun	Complex Business Development Manager	CBDM
noun	Chicago Board Option Exchange	CBOE
noun	Chicago Board of Trade	CBT
noun	Cost Center	CC
noun	Call Center Branch Handled	CCBH
noun	Computer to Computer Facility	CCF
noun	Cost Center Manager	CCM
noun	Client Care Team	CCT
noun	Certificate of Deposit	CD
noun	Client Decisioning Module	CDM
noun	Check Deposit Operations	CDO
noun	Collateralized Debt Obligation	CDO
noun	Contingent Deferred Sales Charge	CDSC
noun	Nominee Name for Depository Trust Company	CEDE
noun	Closed End Funds	CEF
noun	Cost to Equity Ratio	CER
noun	Consulting and Evaluation Service	CES
noun	Corporate Equity Solutions	CES
noun	Coverdell Education Savings Account	CESA
noun	Commodities Future Trading Commission	CFTC
noun	California Tax-Free Daily Income Trust	CFTI
noun	Consulting Group	CG
noun	ADP Cage System	CGA
noun	Consulting Group Advisor	CGA
noun	Consulting Group Capital Markets	CGCM
noun	Citigroup Global Market Inc	CGMI
noun	Citigroup Global Market Inc.	CGMI
noun	Consulting Group Proposal System	CGPS
noun	Clearing House Comparison	CHC
noun	Swiss Franc	CHF
noun	Client Interaction and Account Services	CIAS
noun	Customer Information Control System	CICS
noun	Center for Investment Excellence	CIE
noun	Foreign Securities Identifier	CINS
noun	Customer Identification Program	CIP
noun	Card issuance update	CIU
noun	Consumer Literature Fulfillment	CLF
noun	Collateralized Mortgage Applications	CMA
noun	Cash Management Engagement	CME
noun	Collateralized Mortgage Obligations	CMO
noun	Cage Management System	CMS
noun	Coverage Management Tool	CMT
noun	Cross Trade	CMU
noun	Offshore Chinese Renminbi	CNH
noun	Continuous Net System	CNS
noun	Close of Business	COB
noun	Change of Contact Information	COCI
noun	Cash on Delivery	COD
noun	Computer Output to Laser Disk	COLD
noun	Certificate of Participation	COP
noun	Cash on Receipt	COR
noun	Custom Portfolio	CP
noun	Commodity Pool Operator	CPO
noun	Client Profile System	CPS
noun	Central Processing Unit	CPU
noun	Client Reporting and Control	CRC
noun	Central Registration Depository	CRD
noun	Communications Review Group	CRG
noun	Collateral Risk Management	CRM
noun	Customer Relationship Management	CRM
noun	Challenge Response Question	CRQ
noun	Client Service Associate	CSA
noun	Consulting Group Services	CSG
noun	Complex Service Manager	CSM
noun	Core Securities and Processing Services	CSPS
noun	Client Service Representative	CSR
noun	Customer Service Representative	CSR
noun	Cashless Exercise	CSX
noun	Commodity Trading Advisor	CTA
noun	Committee on Uniform Security Identification Procedures	CUSIP
noun	Client Web Support	CWS
noun	Czech Republic Koruna	CZK
noun	Debit Account Activity	DAA
noun	Discretionary Account Activity	DAA
noun	Defined Contribution Plans	DCP
noun	Disqualifying Disposition	DD
noun	Demand Deposit Account	DDA
noun	Distribution Calculator	DECI
noun	Document Exception Process	DEP
noun	Discretionary Fee Waiver Allowance.	DFWA
noun	Documents in Process	DIP
noun	Dow Jones Industrial Average	DJIA
noun	Danish Krone	DKK
noun	Data Network Services	DNS
noun	Delivery Order	DO
noun	Delegation of Duty	DOD
noun	Days On Fail	DOF
noun	Designated Order Turnaround	DOT
noun	Dividend Reinvestment Program	DRIP or DRP
noun	Dividend Reinvestment Program	DRP or DRIP
noun	Direct Registration System	DRS
noun	District Service Officer	DSO
noun	Directed Share Program	DSP
noun	Depository Trust Company	DTC
noun	Depository Trust and Clearing Corporation	DTCC
noun	Deficit Tracking System	DTS
noun	Delivery vs Payment	DVP
noun	Deposit Withdrawal at Custodian	DWAC
noun	Dollar Weighted Return	DWR
noun	Buy and Hold	E&H
noun	Exercise and Hold	E&H
noun	Employee Benefit Trust	EBT
noun	Equity Commission Discount	ECD
noun	Express Credit Line	ECL
noun	Eligible Contract Participants	ECP
noun	Enhanced due diligence	EDD
noun	Electronic Delivery of Username and Password	EDUP
noun	Executive Financial Services	EFS
noun	Economic Growth and Tax Relief Act	EGTRRA
noun	Europe Middle East and Africa	EMEA
noun	Europe, Middle East, and Africa	EMEA
noun	Event Management Service Bureau	EMSB
noun	Endorsement Missing	END
noun	End of Day	EOD
noun	End of Life	EOL
noun	External Processing Code	EPC
noun	Electronic Pool Notification	EPN
noun	Entitlements Provisioning and Reporting	EPR
noun	Enhancement Request Form	ERF
noun	Employment Retirement Income Security Act	ERISA
noun	Employee Stock Option Plan	ESOP
noun	Employee Stock Purchase Plan	ESPP
noun	Electronic Trade Confirmation	ETC
noun	Exchange Traded Funds	ETF
noun	End User Computing	EUC
noun	Euro	EUR
noun	Exercise and Hold	EXH
noun	Financial Advisor	FA
noun	Foreign Account Control Group	FACG
noun	Fiduciary Asset Management	FAM
noun	Financial Advisor Marketing Center	FAMC
noun	Former Advisor Program	FAP
noun	Financial Advisor Revenue Sponsor	FARS
noun	Fast Automatic Stock Transfer	FAST
noun	Foreign Account Tax Compliance Act	FATCA
noun	Financial Consultant	FC
noun	Financial Control Group	FCG
noun	First Data Corporation	FDC
noun	Federal Deposit Insurance Corporation	FDIC
noun	First Data Resource	FDR
noun	Field Employee Notification and Information Exchange	FENIX
noun	Front End Sales Charge	FESC
noun	First Fund Distributors Inc.	FFDI
noun	Foreign Financial Institution	FFI
noun	Federal Home Loan Mortgage Corp	FHLMC
noun	Fixed income	FI
noun	First In First Out	FIFO
noun	Financial Industry Number	FIN
noun	Financial Industry Regulatory Authority	FINRA
noun	Financial Management Account	FMA
noun	Fair Market Value	FMV
noun	Fill or Kill	FOK
noun	Federal Open Market Committee	FOMC
noun	Fleet Post Office	FPO
noun	Fiduciary Services	FS
noun	Field Services Group	FSG
noun	Fund Solution Account	FSTA
noun	Field Services Unit	FSU
noun	Full Time Employee	FTE
noun	File Transfer Infrastructure	FTI
noun	File Transfer Protocol	FTP
noun	Funds Transfer System	FTS
noun	Funds Processing	FUN
noun	Foreign Exchange	FX
noun	For Your Information	FYI
noun	Great British Pence	GBP
noun	Great British Pounds	GBP
noun	Goals Based Wealth Management	GBWM
noun	Generation and generation of files	GEN
noun	Generation and/or generation of files	GEN
noun	Generation or generation of files	GEN
noun	Global Investment Committee	GIC
noun	Global Identification	GID
noun	Government National Mortgage Association	GNMA
noun	General Obligation	GO
noun	Group Policy Object	GPO
noun	Government Securities Clearing Corporation	GSCC
noun	Global Stock Plan Services	GSPS
noun	Good-Til-Canceled	GTC
noun	Graphical User Interface	GUI
noun	Global Wealth Management Group	GWMG
noun	Home Equity Line of Credit	HELOC
noun	Household	HH
noun	Householding	HH
noun	Hong Kong Dollar	HKD
noun	Buy and Hold	HLD
noun	High net worth	HNW
noun	High Risk Jurisdiction	HRJ
noun	High Risk Transfer	HRT
noun	Household Transfer Application	HTA
noun	International Account Control	IAC
noun	International Bank Account Number	IBAN
noun	Investment Banking Division	IBD
noun	Image Cash Letter	ICL
noun	Investment Consulting Services	ICS
noun	Institutional Delivery	ID
noun	Morgan Stanley IDEAS	IDEAS
noun	Interactive Data Services Incorporated	IDSI
noun	Items for Attention	IFA
noun	Indian Financial Systems Code	IFSC
noun	Individual Investor's Group	IIG
noun	Israeli Shekel	ILS
noun	Instructor led Training	ILT
noun	Instructor-led Training	ILT
noun	Instructorled Training	ILT
noun	Institutional Money Markets	IMM
noun	Inventory Management System	IMS
noun	Investment Management Services	IMS
noun	International Individual Account	IND
noun	Immediate or Cancel	IOC
noun	Indication of Interests	IOI
noun	Interested Party User	IP user
noun	Initial Public Offering	IPO
noun	Integrated Payment System	IPS
noun	Initial Price Thought	IPT
noun	Individual Retirement Account	IRA
noun	Image Replacement Document	IRD
noun	Image Replacement Documents	IRD
noun	Interest Rate Management Application	IRMA
noun	Item Reference Number	IRN
noun	Initial rate of Return	IRR
noun	Internal Revenue Service	IRS
noun	Investment Services Fee	ISF
noun	Institutional Services Group	ISG
noun	International Security Identification Number	ISIN
noun	Incentive Stock Option	ISO
noun	International Stock Plan Service	ISPS
noun	International Stock Purchase Plan	ISPS
noun	International Toll Free	ITF
noun	International Toll Free Telephone Service	ITF
noun	International Toll-Free	ITF
noun	International Toll-Free Telephone Service	ITF
noun	Morgan Stanley Tax-Free Daily Income Trust	ITFI
noun	Interactive Voice Response	IVR
noun	Invest with Rewards	IwR
noun	Journal Automated Workstation System	JAWS
noun	Job Control Language	JCL
noun	Joint Production Number	JPN
noun	Japanese Yen	JPY
noun	Joint Tenant with Rights of Survivorship	JRS
noun	Joint Tenants	JT
noun	Joint Tenants in Common	JT-TIC
noun	Joint Tenants With Rights of Survivorship	JT-WROS
noun	Joint Tenants	JTTEN
noun	Knowledge Based Authentication	KBA
noun	Key Performance Indicator	KPI
noun	Know Your Customer	KYC
noun	Liquidity Access Line	LAL
noun	Local Area Network	LAN
noun	Legal and Compliance Division	LCD
noun	Lightweight Directory Access Protocol	LDAP
noun	Living Disaster Recovery Planning System	LDRPS
noun	London Interbank Offered Rate	LIBOR
noun	Last In First Out	LIFO
noun	Limited Liability Company	LLC
noun	Leave of Absence	LOA
noun	Letter of Authorization	LOA
noun	Liquidation of Assets	LOA
noun	Letter of Intent	LOI
noun	Limited Purpose Account	LPA
noun	Local Plan Administrator	LPA
noun	Legacy Smith Barney	LSB
noun	Local Signature Page	LSP
noun	Limited Trading Authority	LTA
noun	Long Term Incentive Plan	LTIP
noun	Logical Unit	LU
noun	Mergers and Acquisitions	M&A
noun	Master Account View	MAC
noun	Moving Average Convergence/Divergence	MACD
noun	Mobility Autonomous Driving and Electrificatio	MADE
noun	Shared Mobility Autonomous Driving and Electrificatio	MADE
noun	Managed Advisory Portfolio Solutions	MAPS
noun	Morgan Stanley Application Process	MAPS
noun	Mortgage Backed Securities	MBS
noun	Mortgage Backed Securities Clearing Corporation	MBSCC
noun	Mainframe Dual Host	MDH
noun	Miscellaneous Delivery Order	MDO
noun	Multi-Dimension System	MDS
noun	Mutual Fund Institutional Money	MFIM
noun	Mutual Fund Master Security Description	MFMSD
noun	Mutual Funds Operations	MFO
noun	Mutual funds order entry	MFOE
noun	Mutual Funds Routing System	MFRS
noun	Magnetic Ink Character Recognition	MICR
noun	Management Information Systems	MIS
noun	Money Market Fund	MMF
noun	Material Non-public Information	MNPI
noun	Mexican Peso	MNX
noun	Market on Close	MOC
noun	Multi Product Pattern Alert	MPPA
noun	Mobile Remote Deposit Capture	MRDC
noun	Margin New Customer Account Inquiry	MRGI
noun	Margin Customer Account Inquiry	MRGN
noun	Message Routing System	MRS
noun	Morgan Stanley	MS
noun	Morgan Stanley Active Directory	MSAD
noun	Master Security Description	MSD
noun	MSD Online Inquiry System	MSDI
noun	MSD Online Inquiry Only	MSDQ
noun	Morgan Stanley Online	MSO
noun	Morgan Stanley Private Bank, National Association	MSPBNA
noun	Municipal Securities Rulemaking Board	MSRB
noun	Manual Work Items	MWI
noun	National Automated Clearing House Association	NACHA
noun	Name and Address Inquiry	NANQ
noun	New Account Opening Document Processing	NAODP
noun	National Association of Securities Dealers	NASD
noun	National Association of Securities Dealers	NASD 
noun	National Association of Securities Dealers Automated Quotation	NASDAQ
noun	Net Asset Value	NAV
noun	National Best Bid and Offer	NBBO
noun	National Branch Services	NBS
noun	National Credit Union Administration	NCUA
noun	Non-Discretionary Account	NDA
noun	Nondiscretionary Account	NDA
noun	Open a New Batch	NEWB
noun	National Insurance Producer Registry	NIPR
noun	National Market Security	NMS
noun	National New Accounts	NNA
noun	Notice of Order Execution	NOE
noun	Norwegian Krone	NOK
noun	Everyone except North and South America	Non-NSA
noun	Non Stock Plan Services	Non-SPS
noun	Non-Stock Plan Services	Non-SPS
noun	Non-Profit Organization	NPO
noun	Net Present Value	NPV
noun	Non-Qualified Stock Option	NQ
noun	Nonqualified Stock Option	NQ
noun	Non Qualified Stock Option	NQSO
noun	Non-Qualified Stock Option	NQSO
noun	Nonqualified Stock Option	NQSO
noun	Non-Resident Alien	NRA
noun	Nonresident Alien	NRA
noun	National Registration System	NRS
noun	North and South America	NSA
noun	National Securities Clearing Corporation	NSCC
noun	Non Sufficient Funds	NSF
noun	Non-Sufficient Funds	NSF
noun	No Transaction Fee	NTF
noun	New York Municipal Money Market	NTFI
noun	Non Traditional Investments	NTI
noun	Non transferrable security	NTS
noun	Non-transferrable security	NTS
noun	Nontransferrable security	NTS
noun	Non Transferable Securities System	NTSS
noun	Non-Transferable Securities System	NTSS
noun	NonTransferable Securities System	NTSS
noun	New York Futures Exchange	NYFE
noun	New York Stock Exchange	NYSE
noun	New Zealand Dollar	NZD
noun	Outside Business Interest	OBI
noun	Office of the Comptroller of the Currency	OCC
noun	Options Clearing Corporation	OCC
noun	Order Entry	OE
noun	Operations Expenditure Review Committee	OERC
noun	Office of Foreign Assets Control	OFAC
noun	Open Financial Exchange	OFX
noun	Owner Generated Activity	OGA
noun	Original Issue Discount	OID
noun	Online Award Acceptance	OLAA
noun	Online Grant Acceptance	OLGA
noun	Online Trading Services	OLTS
noun	Order Matching File	OMF
noun	Order Matching System	OMS
noun	Ordinary Share	ORD
noun	Over the Counter	OTC
noun	Over the Counter Market	OTC
noun	Over-the-Counter Market	OTC
noun	One Time Passcode	OTP
noun	Office of Thrift Supervision	OTS
noun	Purchase and Sale	P&S
noun	Process Administrator	PA
noun	Professional Alliance Group	PAG
noun	Periodic Accumulated Income Distribution	PAID
noun	Pending Account Number	PAN
noun	Portfolio Advisory Services	PAS
noun	Portfolio Architect Accounts	PATA
noun	Private Banking Group	PBG
noun	Personal Computer	PC
noun	Public Complaints Commission	PCC
noun	Premier Cash Management	PCM
noun	Political Contributions Tracking	PCT
noun	Portable Document Format	PDF
noun	Preset Diversification Program	PDP
noun	Program Development Exchange	PDX
noun	Policy Exceptions and Personal Equipment system	PEPE
noun	Personal Environment Trading	PET
noun	Personal Financial Management	PFM
noun	Product Group Services	PGS
noun	Philadelphia Stock Exchange	PHLX
noun	Personally Identifiable Information	PII
noun	Payment in Kind	PIK
noun	Personal Identification Number	PIN
noun	Portfolio Management	PM
noun	Project Management Office	PMO
noun	Power of Attorney	POA
noun	Payment on Delivery	POD
noun	Pension Protection Act	PPA
noun	Private Placement Life Insurance	PPLI
noun	Periodic Payment System	PPS
noun	Unacceptable use for Participant	PPT
noun	Private Placement Variable Annuity	PPVA
noun	Plastic Same Day	PSC
noun	P&S Input Pending Trades	PSIPT
noun	Purchase and Sales	PSO
noun	Public Switched Telephone Network	PSTN
noun	Participant's Trade Corporation	PTC
noun	Portfolio Performance System	PTF
noun	Platform Transformation Initiative	PTI
noun	Principal Trade Review	PTR
noun	Personal Unique Identification	PUID
noun	Principle Value	PV
noun	Private Wealth Advisor	PWA
noun	Power Word Fortitude	PWF
noun	Private Wealth Management	PWM
noun	Quality Control Packet	QCP
noun	Qualifying Disposition	QD
noun	Qualified Domestic Relations Order	QDRO
noun	Quarterly Performance Reports	QPR
noun	Quick Reference Card	QRC
noun	Qualified Retirement Plan	QRP
noun	Quality Service Approver	QSA
noun	Qualified Supervisory Delegate	QSD
noun	Questionable Trade	QT
noun	Remote Access Control Facility ID	RACF ID
noun	Revenue Anticipation Note	RAN
noun	Royal Bank of Canada	RBC
noun	Required Beginning Date	RBD
noun	Receive Balance Order	RBO
noun	Required Death Distribution	RDD
noun	Retirement Distribution System	RDS
noun	Registration ID	Reg
noun	Registration ID	Reg ID
noun	Reorganization Inquiry	REIQ
noun	Real Estate Investment Trust	REIT
noun	Real Estate Mortgage Investment Conduit	REMIC
noun	Registered Investment Advisor	RIA
noun	Regulated Investment Company	RIC
noun	Retail Inventory Management	RIM
noun	Reduced Instruction Set Computer	RISC
noun	Returned Item Unit	RIU
noun	Remote	RJE
noun	Required Minimum Distribution	RMD
noun	Required Minimum Distributions	RMD
noun	Report Management Distribution System	RMDS
noun	Registered non negotiable	RNN
noun	Registered non-negotiable	RNN
noun	Registered nonnegotiable	RNN
noun	Rights of Accumulation	ROA
noun	Registered Options Principal	ROP
noun	Rate of Return	ROR
noun	Rejected Option Trade Notice	ROTN
noun	Rest of World	ROW
noun	Retirement Plan Manager	RPM
noun	Remittance Processing Pipeline	RPP
noun	Registered Representative	RR
noun	Regional Risk Officer	RRO
noun	Risk Self Assessment	RSA
noun	Risk Self-Assessment	RSA
noun	Risk Self Assessment Adaptive Authentication	RSAAA
noun	Risk Self-Assessment Adaptive Authentication	RSAAA
noun	Relative Strength Index	RSI
noun	Restricted Stock Shares	RSS
noun	Retirement Support Services	RSS
noun	Restricted Stock Unit	RSU
noun	Real Time Accounting	RTA
noun	Real Time Alerts	RTA
noun	Real-Time Accounting	RTA
noun	Realtime Accounting	RTA
noun	Retirement Transfer Type	RTT
noun	Retirement Transfer Tracking	RXT
noun	Settlement Date	S/D
noun	Sales Assistant	SA
noun	San Antonio Branch Handled	SABH
noun	Single Advisory Contract	SAC
noun	State Administration of Foreign Exchange	SAFE
noun	Stock Administration Plan Processing Handling Inquiry Reporting Execution	SAPPHIRE
noun	Stock Appreciation Right	SAR
noun	Salary Reduction Simplified Employee Pension	SAR-SEP
noun	San Antonio Contact Center	SAT
noun	Securities Based Lending	SBL
noun	Stock Clearing Corporation of Philadelphia	SCCP
noun	Strategic Client Reporting	SCR
noun	Same Day Funds Settlement	SDFS
noun	Systems Development Life Cycle	SDLC
noun	Service Delivery Manager	SDM
noun	Self Directed Retirement Account	SDRA
noun	Securities and Exchange Commission	SEC
noun	Security Architecture	SecArch
noun	Swedish Krona	SEK
noun	Securities Exchange Medallion Program	SEMP
noun	Simplified Employee Pension	SEP
noun	Saving for College	SFC
noun	Singapore Dollar	SGD
noun	Shareholder Accounting	SHAC
noun	Securities Industry Association	SIA
noun	Securities Industry Automation Corporation	SIAC
noun	Securities Industry Automation Corporation Tomorrow	SIAT
noun	Securities Information Center	SIC
noun	Standing Instructions Database	SID
noun	Stock Record History	SINH
noun	Stock Record Inquiry	SINR
noun	Systematic Investment Plan	SIP
noun	Securities Investor Protection Corporation	SIPC
noun	Service Level Agreement	SLA
noun	Statement Linked Group	SLG
noun	Separately Managed Account	SMA
noun	Separately Managed Accounts	SMA
noun	Special Memorandum Account	SMA
noun	System Modification Program/Extended	SMP/E
noun	Sign Off from Security System	SNOF
noun	Sign On to Security System	SNON
noun	Segregation of Duties	SOD
noun	Stock Option Employee ID	SOE ID
noun	Stock Option System	SOPT
noun	System of Record	SOR
noun	Service Professional	SP
noun	Signature Page	SP
noun	Security Processing Application	SPA
noun	Scratch Pad	SPAD
noun	Stock Plan Connect	SPC
noun	StockPlan Connect	SPC
noun	Special Instruction Code	SPIN
noun	Special Payment Order	SPO
noun	Special Products Payment System	SPPS
noun	Senior Complex Risk Officer	Sr. CRO
noun	State Registration Expense Platform	SREP
noun	Service Request Inquiry	SRI
noun	Senior Registered Options Principal	SROP
noun	System Required Specification	SRS
noun	Service Review Unit	SRU
noun	Stock Settled SAR	SSAR
noun	Stock-Settled SAR	SSAR
noun	Social Security Number	SSN
noun	Single Sign On	SSO
noun	System Service Request	SSR
noun	Settled stock Appreciation Rights	SSRs
noun	Securities Transfer Association Medallion Program	STAMP
noun	Sell To Cover	STC
noun	State Tuition Plan	STP
noun	Straight Through Processing	STP
noun	Separate Trading of Registered Interest and Principal Securities	STRIPS
noun	Share Value Plan	SVP
noun	Saved Wire Beneficiary	SWB
noun	Society for Worldwide Interbank Financial Telecommunication	SWIFT
noun	Systematic Withdrawal Investment Plan	SWIP
noun	Systematic Withdrawal Plan	SWP
noun	Treasures or Trading Day	T
noun	Travel and Expense	T&E
noun	Trade Date Plus Two Business Days	T+2
noun	Trade Date	T/D
noun	Trailing twelve months	T12
noun	Tax Anticipation Note	TAN
noun	Trade Activity Report	TAR
noun	N/A	TASQ
noun	Transfer Control Protocol Internet Protocol	TCP/IP
noun	Transfer Control Protocol/Internet Protocol	TCP/IP
noun	Task Checklist Questions	TCQ
noun	Taxpayer Equity Fiscal Responsibility Act	TEFRA
noun	TERM ID Inquiry	TERM
noun	Tape Input File	TIF
noun	Transfer Initiation Form	TIF
noun	Tax Identification Number	TIN
noun	Transition Management Services	TMS
noun	Transfer of Account	TOA
noun	Transfer on Death	TOD
noun	Top Operating Model	TOM
noun	Targeted Percentage of Volume	TPOV
noun	Third Party Verbal	TPV
noun	Tax Reform Act	TRA
noun	Report Directory	TRAC
noun	Travel Rule Indicator	TRI
noun	Transaction Identifier	TRID
noun	Time Weighted Return	TWR
noun	Uniform Books and Records	UBR
noun	Uncollected Funds	UCF
noun	User Defined	UD
noun	Uniform Gift to Minors Act	UGMA
noun	Ultra High Net Worth	UHNW
noun	Universal International Free phone Number	UIFN
noun	Unit Investment Trust	UIT
noun	Unified Managed Account	UMA
noun	United States Dollar	USD
noun	Uniform Transfers to Minors Act	UTMA
noun	User Experience	UX
noun	Virtual Branch Office	VBO
noun	Verbal Distribution Authentication	VDA
noun	Voluntary Employees' Beneficiary Association Trust	VEBA
noun	Versatile Investment Program	VIP
noun	Voice Response Unit	VRU
noun	Virginia State College Plan	VSCP
noun	Variable Standing Instruction	VSI
noun	Sales vs. Purchase	VSP
noun	Vendor Support Systems	VSS
noun	Virtual Training Team	VTT
noun	Variable Weighted Average Price	VWAP
noun	Wealth Advisory Associate	WAA
noun	Wide Area Network	WAN
noun	WorldLink Check	WC
noun	Workflow Dashboard	WFD
noun	Widely Held Fixed Investment Trusts	WHFIT
noun	When Issued	WI
noun	WorldLink	WL
noun	Written Supervisory Procedure	WSP
noun	Withhold to Cover	WTC
noun	WorldLink Wires	WW
noun	Cross Reference	XREF
noun	Patriot Act Entity	PAE
noun	Patriot Act Address	PAA
noun	Unified Estate	UE
noun	Unified Estate	u.e
noun	ct-w4p	CT
noun	Account Opening	A.O
noun	Account Opening	AO
noun	3210 alert	3210s
noun	3210 alert	fina
noun	3210 alert	finra
noun	3d	32d
noun	3d	3dao
noun	3d	in3d
noun	3d	pti
noun	3d	tpi
noun	401k account	401
noun	401k account	401a
noun	401k account	401k
noun	401k account	401kin
noun	401k account	401kplan
noun	401k account	401l
noun	403b account	403
noun	403b account	403b
noun	403b account	irb
noun	407 approval letter	407
noun	457f account	457
noun	457f account	457f
noun	529 account	529
noun	529 account	529s
noun	529 account	in529
noun	529 college savings plan application and client agreement	529app
noun	529 saving for college account	sfc
noun	529 state tuition plan account	stp
noun	72t distribution	72t
noun	72t distribution	72td
noun	72t distribution	72ts
noun	acat cancellation form	atsacatc
noun	account application and client agreement	aaca
noun	account inception	incept
noun	account inception	incepted
noun	account inception	inception
noun	account inception and update tool	aiut
noun	account linking service	linking
noun	account management portal	amp
noun	account name	name
noun	account number	acct#
noun	account number	number
noun	account plating	eplating
noun	account plating	pate
noun	account plating	plaiting
noun	account plating	plaitng
noun	account plating	plat
noun	account plating	plate
noun	account plating	plated
noun	account plating	platin
noun	account plating	plating
noun	account plating	platings
noun	account plating	platting
noun	account statement	statement
noun	account statement	statements
noun	account swing	swing
noun	account transfer form	acats
noun	account transfer form	atsatff
noun	account type change	aatc
noun	account type change	act
noun	account type change	atc
noun	account type change	change
noun	account type change	changed
noun	account type change	changing
noun	account type change	convert
noun	account type change	converting
noun	account type change	make
noun	account type change	natc
noun	account type change	transfer
noun	account type change	transition
noun	account type change	turn
noun	account type change	update
noun	account type change	xfr
noun	acknowledgement by person employed by a broker-dealer finra or an sro letter	sro
noun	address	addres
noun	address	theaddress
noun	affidavit of domicile and debts	aod
noun	affidavit of domicile and debts	avd
noun	affidavit of domicile and debts	domicile
noun	alert	alet
noun	alert	elrt
noun	alert	malert
noun	alternative investments advisory account	aia
noun	amortization	amortizing
noun	amortization	amotirization
noun	annuity	annu9ity
noun	annuity	annuities
noun	annuity	annuityy
noun	anti money laundering program	aml
noun	authorized individual	a.i
noun	authorized individual	ais
noun	authorized individual	allai
noun	authorized individual	auth
noun	authorized individual	authorized
noun	authorized individual	signer
noun	authorized individual	signers
noun	authorized individual	signor
noun	authorized persons and enabling resolutions for corporations form	caa
noun	automated clearing house transaction	ach
noun	automated customer account transfer	acat
noun	automated customer account transfer	acated
noun	automated customer account transfer	acating
noun	automated customer account transfer	acats
noun	automated customer account transfer	achat
noun	automated required minimum distribution	autormd
noun	automated required minimum distribution	drma
noun	automatic sweep	sweep
noun	automatic sweep	sweeping
noun	automatic sweep	swept
noun	b notice	bnotice
noun	backup withholding	withholding
noun	balance	balances
noun	bank deposit program	bdp
noun	bank middle market custody account	bfc
noun	bank middle market dvp account	bdc
noun	banking service	banking
noun	basic securities account	bsa
noun	beneficial owner	b.o.
noun	beneficial owner	beneowner
noun	beneficial owner	bos
noun	beneficial owner certification	boc
noun	beneficial owner certification	ubo
noun	beneficiary	beanies
noun	beneficiary	been
noun	beneficiary	ben
noun	beneficiary	bene
noun	beneficiary	bened
noun	beneficiary	benef
noun	beneficiary	benefciary
noun	beneficiary	benefiacries
noun	beneficiary	benefiaries
noun	beneficiary	beneficary
noun	beneficiary	beneficciaries
noun	beneficiary	beneficiares
noun	beneficiary	beneficiaries
noun	beneficiary	beneficiaries'
noun	beneficiary	beneficiaryies
noun	beneficiary	beneficiarys
noun	beneficiary	beneficicary
noun	beneficiary	benefiiary
noun	beneficiary	benefiicary
noun	beneficiary	benefiociaries
noun	beneficiary	beneies
noun	beneficiary	beneificiaries
noun	beneficiary	benes
noun	beneficiary	benficiaries'
noun	beneficiary	beni
noun	beneficiary	benies
noun	beneficiary	benne
noun	beneficiary	bennie
noun	beneficiary	bennies
noun	beneficiary	bennis
noun	beneficiary	bens
noun	beneficiary	bne
noun	beneficiary	rbene
noun	beneficiary	thebene
noun	beneficiary's full name	name
noun	blanket approval letter	bal
noun	blanket approval letter	blanket
noun	block buys trading only red flag	13a
noun	blp - business other account	gbo
noun	blp - charity organization account	gwc
noun	blp - incorporated account	gbi
noun	blp - individual account	gwi
noun	blp - joint account	gwj
noun	blp - listed company account	gwl
noun	blp - personal holding company account	gwp
noun	blp - sole prop account	gbs
noun	blp - trust account	gwt
noun	blp account	blp
noun	branch	brach
noun	branch	branc
noun	branch	branched
noun	branch	branches
noun	branch	branchs
noun	branch	brnach
noun	branch	brnaches
noun	branch office managers	bom
noun	branch office support system	boss
noun	branch workflow	branchworkflow
noun	branch workflow	bwf
noun	branch written supervisory procedure	wsp
noun	branch-issued check	checks
noun	broker dealer	b.d
noun	broker dealer	bor
noun	broker dealer	broker
noun	broker dealer	stockbroker
noun	broker identification number	bin
noun	brokerage account	brokerage
noun	business active assets account	baaa
noun	business active assets account	bus
noun	business active assets account	business
noun	business active assets account	bussn
noun	business basic securities account	bbsa
noun	business retirement account	arp
noun	business retirement account	brp
noun	business retirement account	qpr
noun	business retirement account	qps
noun	business retirement account	qrp
noun	business retirement account	qrps
noun	business retirement account	qualified
noun	business service manager	besm
noun	business service manager	bsm
noun	business service manager	bsn
noun	business service manager	manager
noun	captive insurance account	captive
noun	case management request	case
noun	case management request	cases
noun	cash	money
noun	cash account	cash
noun	cash on delivery	cod
noun	cashless exercise account	csx
noun	center for investment excellence	cie
noun	central review unit	cru
noun	certificate	cert
noun	certificate	certificates
noun	certificate	certification
noun	certificate of deposit	cds
noun	cg advisor loa consenting to principal trading	cgploapt
noun	charitable remainder trust account	crt
noun	charitable remainder trust account	cruts
noun	check	cehcks
noun	check	checck
noun	check	checking
noun	check	checks
noun	check	chekc
noun	check	paycheck
noun	check writing privilege	checkwriitng
noun	check writing privilege	checkwriting
noun	check writing privilege	checkwritting
noun	child	children
noun	child support enforcement - allow trading red flag	5c1
noun	child support enforcement - block all transactions red flag	5c2
noun	child support enforcement- liquidation orders only red flag	5c3
noun	city	london
noun	city	nyc
noun	civil dispute - allow trading red flag	5d1
noun	civil dispute - block all transactions red flag	5d2
noun	civil dispute - liquidation orders only red flag	5d3
noun	class action query system	caq
noun	client advisory center	cac
noun	client disclosure document	cdd
noun	client profile	profile
noun	client profile system	cps
noun	client service associate	csa
noun	client web support	cws
noun	closed jurisdiction with loan red flag	15b
noun	commission and processing fees	commission
noun	commission and processing fees	commissions
noun	commodity	commodities
noun	communications review center	crc
noun	community property account	com
noun	complex risk officer	cro
noun	complex risk officer	cros
noun	conference call	teleconference
noun	connecticut higher education trust - advisor plan account	chet
noun	connecticut w-4 p form	ct24p
noun	connecticut w-4 p form	ct4p
noun	connecticut w-4 p form	ctw4p
noun	consolidated year end summary statement	cyess
noun	consulting and evaluation services account	ces
noun	consulting group advisor account	cga
noun	contingent beneficiary	contingent
noun	contingent beneficiary	contingents
noun	contingent deferred sale charge	cdsc
noun	contrafirm	contra
noun	contribution	conbtrib
noun	contribution	conbtribution
noun	contribution	congtrib
noun	contribution	cont
noun	contribution	contib
noun	contribution	contibs
noun	contribution	contirb
noun	contribution	contirbs
noun	contribution	contirbution
noun	contribution	contirbutions
noun	contribution	contirbuton
noun	contribution	contr
noun	contribution	contrb
noun	contribution	contrbution
noun	contribution	contri
noun	contribution	contrib
noun	contribution	contribn
noun	contribution	contribs
noun	contribution	contribtuion
noun	contribution	contribtuiosn
noun	contribution	contribtution
noun	contribution	contribuitn
noun	contribution	contribuiton
noun	contribution	contribute
noun	contribution	contributing
noun	contribution	contributioin
noun	contribution	contributions
noun	contribution	contributios
noun	contribution	contributons
noun	contribution	contris
noun	contribution	ocntributon
noun	contribution coding	coding
noun	contribution deadline	deadline
noun	contribution deadline	deadlines
noun	contribution limit	limit
noun	contribution limit	limits
noun	contribution limit	llimit
noun	contribution limit	max
noun	contribution recharacterization	rech
noun	contribution recharacterization	rechacterization
noun	contribution recharacterization	rechar
noun	coogan trust account	coogan
noun	corporation account	corp
noun	corporation account	corporaiton
noun	corporation account	corporate
noun	corporation account	corporation
noun	corporation account	corporations
noun	corporation account	corps
noun	corporation account	inc
noun	country	austrailia
noun	country	australia
noun	country	bahamas
noun	country	belgium
noun	country	bvi
noun	country	canada
noun	country	canadian
noun	country	china
noun	country	countries
noun	country	egypt
noun	country	france
noun	country	germany
noun	country	guam
noun	country	guatemala
noun	country	guyana
noun	country	ireland
noun	country	israel
noun	country	japan
noun	country	mexico
noun	country	monaco
noun	country	pakastan
noun	country	scotland
noun	country	singapore
noun	country	spain
noun	country	switerland
noun	country	thailand
noun	country	u.k
noun	country	u.s
noun	country	u.s.
noun	country	usa
noun	country	venezula
noun	court order - allow trading red flag	5a1
noun	court order - block all transactions red flag	5a2
noun	court order - liquidation orders only red flag	5a3
noun	coverdell education savings account	cesa
noun	coverdell education savings account	converdell
noun	coverdell education savings account	coverdell
noun	coverdell education savings account	coverdells
noun	coverdell education savings account	esa
noun	cure document	cure
noun	cure document	doc
noun	currency	dollar
noun	currency	dollars
noun	currency	euro
noun	currency	euros
noun	currency	pound
noun	cusip number	cusip
noun	custodian	cus
noun	custodian	cusodian
noun	custodian	custo
noun	custodian	custodians
noun	custodian	custodina
noun	custodian account	cus
noun	custodian account	cust
noun	custodian account	custod
noun	custodian account	custodial
noun	custodian account	custodian
noun	customer identification program	cip
noun	customer identification program process	ciped
noun	data quality - block all activity red flag	16b
noun	data quality - restrict buy trades only red flag	16d
noun	data quality - restrict deposits and withdrawals red flag	16c
noun	date of birth	birthdate
noun	date of birth	birthday
noun	date of birth	d.o.b
noun	date of birth	dob
noun	date of death	dod
noun	daughter	daughters
noun	deadline	deadlines
noun	death certificate	dct
noun	death distribution	death
noun	death distribution	deathdist
noun	death distribution	distribution
noun	debit card	debit
noun	debit card	debti
noun	deceased account owner red flag	deceasd
noun	deceased client	dec
noun	deceased client	decadent
noun	deceased client	decease
noun	deceased client	deceased
noun	deceased client	decedant
noun	deceased client	decedent
noun	deceased client	decedents
noun	deceased client	decednt
noun	deceased client	decendents
noun	deceased client	decent
noun	deceased client	decents
noun	deductible asset	deductible
noun	default beneficiary order	default
noun	delivery versus payment	dvp
noun	deposit	deosited
noun	deposit	depoit
noun	deposit	deposited
noun	deposit	deposits
noun	deposit	deposti
noun	depository trust & clearing corporation	dtcc
noun	depository trust check	dtc
noun	depository trust company transfer	dtc
noun	direct rollover	ddro
noun	directed share business account	dsb
noun	directed share program	dsp
noun	disbursement	disburse
noun	disbursement	disbursed
noun	disbursement	disbursements
noun	disbursement	disbursing
noun	disbursement	disbusement
noun	disbursement restriction -lending operations red flag	12a
noun	discount rate	discount
noun	discretionary fee waiver allowance	dfwa
noun	display account restriction red flag	11a
noun	distribution	diddist
noun	distribution	dis
noun	distribution	dis.t
noun	distribution	disribution
noun	distribution	dist
noun	distribution	dist.
noun	distribution	distibituion
noun	distribution	distibution
noun	distribution	distirbuiton
noun	distribution	distirbution
noun	distribution	distirubtion
noun	distribution	distrbution
noun	distribution	distribtuion
noun	distribution	distribuition
noun	distribution	distribuiton
noun	distribution	distribut9ion
noun	distribution	distribute
noun	distribution	distributin
noun	distribution	distributionj
noun	distribution	distributionrequest
noun	distribution	distributions
noun	distribution	distribution_zip_
noun	distribution	distributoin
noun	distribution	distributoion
noun	distribution	distributrion
noun	distribution	distribuytion
noun	distribution	distriibution
noun	distribution	distriubiton
noun	distribution	distrobution
noun	distribution	distrubution
noun	distribution	dists
noun	distribution	distz
noun	distribution	ditribution
noun	distribution	diust
noun	distribution	dost
noun	distribution	dsitribution
noun	distribution	hdistribution
noun	distribution	w;dist
noun	distribution form	distribution
noun	distribution form	distributions
noun	distribution form	distrubition
noun	distribution options	distribution
noun	distribution options	distributions
noun	distribution options	method
noun	distribution options	option
noun	distribution options	options
noun	dividend	div
noun	dividend	dividends
noun	divorce distribution	divorce
noun	divorce distribution	divorces
noun	document sharing automation tool	docshare
noun	document sharing automation tool	docushare
noun	duplicate distribution	duplicated
noun	durable power of attorney full discretionary authorization form	gvd
noun	early dispute resolution department	edr
noun	eauthorization	eauth
noun	edelivery preferences	edelivery
noun	edelivery preferences	edilivery
noun	edelivery preferences	edlivery
noun	electronic funds transfer	eft
noun	email	emailed
noun	email	emails
noun	email	emaoil
noun	email address	eamil
noun	email address	emaail
noun	email address	email
noun	employer identification number	ein
noun	employment information	employment
noun	employment retirement income security act of 1974	erisa
noun	enhanced due diligence process	edd
noun	enhanced due diligence questionnaire	anedd
noun	enhanced due diligence questionnaire	edd
noun	enhanced due diligence questionnaire	eddauthenticated
noun	enhanced due diligence questionnaire	eddd
noun	enhanced due diligence questionnaire	edds
noun	enhanced due diligence questionnaire	edd}ff
noun	enhanced due diligence questionnaire	eed
noun	enhanced due diligence questionnaire	inedd
noun	entitlements provisioning and reporting team	epr
noun	entity name	ameritrade
noun	entity name	autovest
noun	entity name	barney
noun	entity name	bis
noun	entity name	blackstone
noun	entity name	cbm
noun	entity name	chase
noun	entity name	comerica
noun	entity name	ecl
noun	entity name	fidelity
noun	entity name	hewitt
noun	entity name	lincoln
noun	entity name	lionhearted
noun	entity name	m.s
noun	entity name	mccm
noun	entity name	merril
noun	entity name	morgan
noun	entity name	mssb
noun	entity name	name
noun	entity name	penchecks
noun	entity name	penchex
noun	entity name	penco
noun	entity name	pensco
noun	entity name	pfizer
noun	entity name	schwab
noun	entity name	schwabb
noun	entity name	suntrust
noun	entity name	tiaa
noun	entity name	transamerica
noun	entity name	vanguard
noun	entity name	voya
noun	entity name	wells
noun	equal credit opportunity act	ecoa
noun	equity	equities
noun	equity	sec
noun	equity	securities
noun	equity	security
noun	equity	share
noun	equity	shares
noun	equity	shars
noun	equity	stock
noun	equity	stocks
noun	erisa held away exception alert	eha
noun	erisa plan held away account	eha
noun	erisa plan held away account	ehas
noun	erisa plan held away account	erisa
noun	erisa plan held away account	erisas
noun	escheatment operations department	escheatment
noun	esign	design
noun	esign	easign
noun	esign	eign
noun	esign	eisgn
noun	esign	esgn
noun	esign	esigh
noun	esign	esignable
noun	esign	esignature
noun	esign	esigned
noun	esign	esignhow
noun	esign	esigning
noun	esign	esignpacklage
noun	esign	esing
noun	esign	inesign
noun	esign	nesign
noun	esign	package
noun	esign	packet
noun	esign	theesign
noun	estate account	est
noun	estate account	estate
noun	estate beneficiary	esate
noun	estate document	est
noun	excess contribution	exces
noun	excess contribution	excess
noun	excess contribution	excessdue
noun	excess contribution	excuses
noun	excess contribution	exess
noun	excess contribution	exzcess
noun	excess contribution	xcess
noun	exchange traded fund	etf
noun	executor	excurtor
noun	executor	exec
noun	executor	executer
noun	executor	executier
noun	executor	executors
noun	express creditline	ecl
noun	express creditline	sbl
noun	external custodian ira/keogh account	keogh
noun	external power of attorney	external
noun	federal tax withholding	fed
noun	federal tax withholding	federal
noun	fee	fees
noun	fee	wasfee
noun	fiduciary certification	fiduciary
noun	fiduciary certification and trust account agreement	trd
noun	field services unit of legal compliance	fsu
noun	field services unit of legal compliance	legal
noun	financial institution questionnaire	ffi
noun	financial institution questionnaire	fiq
noun	finra 3210 rule	3210
noun	finra 3210 rule	finra
noun	finra 3210 rule	firna
noun	fixed income	bond
noun	fixed income	bonds
noun	fixed income service team	fis
noun	fixed rate note	frn
noun	fixed recurring distribution	recurring
noun	foreign account tax compliance act	fatca
noun	foreign exchange new account summary form	ctmfenas
noun	foreign financial institution	ffi
noun	foreign related entity account	foredign
noun	fraud - allow trading red flag	5f1
noun	fraud - block all transactions red flag	5f2
noun	fraud - liquidation orders only red flag	5f3
noun	fraudulent activity	fraud
noun	fund	funds
noun	funds transfer service enrollment form	fts
noun	funds transfer system	fts
noun	futures account agreement	faa
noun	general partnership account	par
noun	general partnership account	ppartnerhsip
noun	general purpose - international account control red flag	15a
noun	general purpose plus block check or card order red flag	15d
noun	generation skipping trust account	gst
noun	gift	donation
noun	gift	donations
noun	gift	gifts
noun	global currency express	gce
noun	global product screener	gps
noun	government entity account	gov
noun	government issued photo id	gd1
noun	government issued photo id	gdm
noun	government issued photo id	govid
noun	government issued photo id	gvo
noun	grantor	gran
noun	grantor	grantors
noun	grantor	settlor
noun	grantor	trustor
noun	grantor retained trust account	grat
noun	guardian	conservator
noun	guardian	conservators
noun	guardian	conservitor
noun	guardian	gdn
noun	guardian	gua
noun	guardian	guardians
noun	guardian	guardianship
noun	guardianship account	conservator
noun	guardianship account	conservatorship
noun	guardianship account	gua
noun	guardianship account	guard
noun	guardianship account	guardian
noun	guardianship account	guardianship
noun	hard dollar fee - community property account	mcp
noun	hard dollar fee - iro account	mhi
noun	hard dollar fee - joint tenants by the entirety account	mje
noun	hard dollar fee - joint tenants in common account	mjc
noun	hard dollar fee - non-profit organization account	mnp
noun	hard dollar fee - non-profit organization account	npo
noun	hard dollar fee - religious organization account	mpr
noun	hard dollar fee - testamentary trust account	mtt
noun	hard dollar fee - unincorporated association account	mpa
noun	hard dollar fee -guardianship account	mpg
noun	hard dollar fee -incorporated account	mpi
noun	hard dollar fee a/c ind account	mhn
noun	hard dollar fee erisa trust account	mpe
noun	hard dollar fee government agency account	mgv
noun	hard dollar fee limited liability company account	mpl
noun	hard dollar fee living trust account	mpt
noun	hard dollar fee partnership account	mpp
noun	high risk address	hra
noun	householding	househeld
noun	householding	household
noun	householding	householded
noun	householding	households
noun	householding	householing
noun	householding	houshold
noun	householding	housholding
noun	householding department	householding
noun	id	identification
noun	id	ids
noun	in-kind contribution	inkind
noun	in-kind contribution	inkine
noun	in-kind distribution	inkind
noun	incapacitated person	incapacitated
noun	income	earnings
noun	incorporated non-profit organization account	foundation
noun	incorporated non-profit organization account	incorporated
noun	incorporated non-profit organization account	npi
noun	individual 401k account	ikk
noun	individual 401k account	indivk
noun	individual 401k account	indv
noun	individual 401k account	indvk
noun	individual 401k account	indyk
noun	individual active assets account	aaa
noun	individual active assets account	aaaa
noun	individual retirement account	iira
noun	individual retirement account	ir.a
noun	individual retirement account	ira
noun	individual retirement account	ira'
noun	individual retirement account	iras
noun	individual retirement account adoption agreement	ira
noun	individual retirement account distribution form	rdd
noun	inherited ira account	inh
noun	inherited ira account	inherited
noun	inherited ira account	inheritedira
noun	inherited ira account	ira
noun	initial public offering	ipo
noun	insurance product	insurance
noun	insurance ticketing system	its
noun	interest amount	accruedinterest
noun	interest amount	int
noun	interest amount	interest
noun	interest rate	rate
noun	interest rate	rates
noun	interest rate management application	irma
noun	interested party user	interested
noun	internal power of attorney	internal
noun	international account	foreign
noun	international account	int
noun	international account	internantional
noun	international account	international
noun	international account	intl
noun	international account	nra
noun	international client tax certification	gqn
noun	investment club account	clu
noun	investment management services program account	ims
noun	investment policy statement	ips
noun	investment product	investment
noun	investment product	investments
noun	investment services fee	isf
noun	invoice	bill
noun	ira - the morgan stanley roth ira account opening booklet	retirarb
noun	ira designation of beneficiary form	irb
noun	ira outside custodian account	iro
noun	irrevocable life insurance trust account	ilt
noun	irrevocable living trust account	irr
noun	irrevocable living trust account	irrev
noun	irrevocable living trust account	irrevo
noun	irrevocable living trust account	irrevoc
noun	irrevocable living trust account	irrevocable
noun	irrevocable living trust account	irrevocalbe
noun	irrevocable living trust account	irrevocbale
noun	irs form 1099	1099
noun	irs form 1099-q: payments from qualified education programs	1099q
noun	irs form 1099-r	1009r
noun	irs form 1099-r	1099r
noun	irs form 1099-r	1099rs
noun	irs form 5173	5173
noun	irs form 5498	5498
noun	irs form 5498	5498s
noun	is client living indicator	deceased
noun	joint account	ajoint
noun	joint account	jint
noun	joint account	jnt
noun	joint account	join
noun	joint account	joint
noun	joint account	joints
noun	joint account	jpoint
noun	joint account agreement with right of survivorship	jawros
noun	joint production number	jpn
noun	joint tenancy in common account	jtc
noun	joint tenant with rights of survivorship account	jawros
noun	joint tenant with rights of survivorship account	jros
noun	joint tenant with rights of survivorship account	jrs
noun	joint tenant with rights of survivorship account	jrwros
noun	joint tenant with rights of survivorship account	jtwos
noun	joint tenant with rights of survivorship account	jtwros
noun	joint tenant with rights of survivorship account	jtwrs
noun	joint tenant with rights of survivorship account	jtwtros
noun	joint tenant with rights of survivorship account	jwros
noun	joint tenant with rights of survivorship account	wros
noun	joint tenants by the entirety account	ent
noun	joint tenants by the entirety account	entirity
noun	journal	jjournal
noun	journal	jnl
noun	journal	jouirnal
noun	journal	journ
noun	journal	journaled
noun	journal	journaling
noun	journal	journall
noun	journal	journalling
noun	journal	journals
noun	journal	journel
noun	journal	journeled
noun	journal	jrl
noun	journal automated workstation system	jaws
noun	journal contribution	journal
noun	journal entry request	jes
noun	journal entry request	journal
noun	judgment of possession	jop
noun	key controller	keycontroller
noun	know your customer	kyc
noun	lal - business other account	lao
noun	lal - charity not for profit account	lan
noun	lal - corporation account	lac
noun	lal - individual account	lai
noun	lal - joint account	laj
noun	lal - partnership account	lap
noun	lal - personal holding co account	lah
noun	lal - sole prop account	las
noun	lal - trust account	lat
noun	lal account	lal
noun	lal account	lals
noun	lal add account request form	lenlalar
noun	lal certification for partnerships and llcs	lenlalpc
noun	language	english
noun	language	portuguese
noun	last will and testament	lswlts
noun	last will and testament	lwt
noun	legal address	legal
noun	letter of authorization	loa
noun	letter of authorization	loas
noun	letter of authorization	lta
noun	letter of credit	loc
noun	letter of explanation	loe
noun	letter of instruction	loi
noun	letter of intent	loi
noun	letters of guardianship	conservatorship
noun	letters of guardianship	ltrs
noun	letters testamentary	llt
noun	letters testamentary	ltt
noun	letters testamentary	testamentary
noun	lexisnexis	lexis
noun	life tenant account	lif
noun	lifeview portal	lifeview
noun	limited liability company account	lcc
noun	limited liability company account	llc
noun	limited liability company account	llcs
noun	limited liability company account	lle
noun	limited liability company account	pllc
noun	limited liability entity agreement	llc
noun	limited liability entity agreement	lle
noun	limited partnership	lpr
noun	limited partnership	partnership
noun	limited partnership account	limited
noun	limited partnership account	lpr
noun	limited partnership account	partner
noun	limited partnership account	partnership
noun	liquidation of assets	liquidatation
noun	liquidation of assets	liquidate
noun	liquidation of assets	liquidated
noun	liquidation of assets	liquidation
noun	liquidity access line financial statement	lenlalfs
noun	liquidity access line trust certification	lenlaltr
noun	living trust account	aliving
noun	living trust account	liv
noun	living trust account	living
noun	loan	loans
noun	loan	lona
noun	loan account	loan
noun	local signature page	elsp
noun	local signature page	lps
noun	local signature page	lsp
noun	local signature page	lsps
noun	local signature page	signature
noun	local signature page	slp
noun	local signature page	slsp
noun	london interbank offered rate	libor
noun	lump sum distribution	lumpsum
noun	mailing address	mailing
noun	managed account	manage
noun	managed account	managed
noun	managed advisory portfolio solutions	maps
noun	margin	emargin
noun	margin	maring
noun	margin	mrgin
noun	margin account	margin
noun	margin and loan agreement	mla
noun	married	marriage
noun	mexican pesos	mxn
noun	michigan w-4 p form	mi4p
noun	michigan w-4 p form	miw4p
noun	middle markets custody account	mmc
noun	middle markets dvp account	fmm
noun	middle name	middle
noun	minimum advisory fee	minimum
noun	minor beneficiary	minor
noun	minor beneficiary	minors
noun	missing enhanced due diligence red flag	edd
noun	missing politically exposed person info red flag	pep
noun	model	modell
noun	money market fund	mmf
noun	morgan stanley global impact funding trust account	daf
noun	morgan stanley global impact funding trust account	gift
noun	morgan stanley online	mso
noun	morgan stanley online	msol
noun	morgan stanley online	msonline
noun	morgan stanley online	online
noun	morgan stanley online payment	billpay
noun	morgan stanley virtual advisor	msva
noun	mortgage	mortage
noun	msai fraud operations review red flag	17a
noun	mutual fund	fund
noun	mutual fund	funds
noun	mutual fund block red flag	15c
noun	national new accounts red flag	nna_reflags
noun	national securities clearing corporation	nscc
noun	new account opening questionnaire	nao
noun	new account opening tool	3dnao
noun	new account opening tool	3nao
noun	new account opening tool	nao
noun	new account opening tool	naotool
noun	next best action	nba
noun	nna department	nna
noun	nna workflow tools	nna
noun	nna workflow tools	nnatools
noun	non resident alien	nra
noun	non traditional investment	nti
noun	non-custodial ira application and agreement	retnoncu
noun	non-deductible asset	nondeductible
noun	non-probate estate account	int
noun	non-spouse beneficiary	nonspouse
noun	north carolina w-4 p form	nc4p
noun	north carolina w-4 p form	nc4wp
noun	notification	norification
noun	notification	notice
noun	notification	notifications
noun	officer	officers
noun	offshore trust - partnership account	itp
noun	offshore trust - personal investment corporation account	itc
noun	offshore trust account	iti
noun	ok to comply	okcomply
noun	ok to comply	oktocompl
noun	ok to comply	oktocomply
noun	one hundred percent tax withholding	100
noun	option	optiion
noun	option	options
noun	options new account form and client agreement	opa
noun	options new account form and client agreement	options
noun	outside custodian ira retirement plan account	kpo
noun	overseas	internantional
noun	owner	own
noun	owner	owners
noun	owner	ownrer
noun	owner	woner
noun	paperless transfer program	pltp
noun	participant account	partic
noun	participant account	participant
noun	partnership account	partnership
noun	password	otp
noun	patriot act address verification document	paa
noun	patriot act address verification document	paa2
noun	patriot act address verification document	paa2be
noun	patriot act address verification document	paa3
noun	patriot act address verification document	paa4
noun	patriot act address verification document	paa5
noun	patriot act address verification document	paa6
noun	patriot act address verification document	paaa
noun	patriot act address verification document	paapat
noun	patriot act address verification document	paas
noun	patriot act address verification document	pad
noun	patriot act address verification document	ppa
noun	patriot act date of birth verification document	apb
noun	patriot act date of birth verification document	dob
noun	patriot act date of birth verification document	pab
noun	patriot act date of birth verification document	pad
noun	patriot act date of birth verification document	pad3
noun	patriot act date of birth verification document	padob
noun	patriot act date of birth verification document	pob
noun	patriot act entity verification document	entity
noun	patriot act entity verification document	pae
noun	patriot act entity verification document	paet
noun	patriot act entity verification document	spae
noun	patriot act entity verification document	thepae
noun	patriot act social security number verification document	3pas2
noun	patriot act social security number verification document	pas
noun	patriot act social security number verification document	ss4
noun	patriot act social security number verification document	sss
noun	patriot act verification document	pa2
noun	patriot act verification document	pat
noun	payment	paid
noun	payment	pay
noun	payment	payable
noun	payment	payed
noun	payment	paying
noun	payment	payments
noun	payment	pays
noun	payment	pmt
noun	pending account number	pan
noun	pending account number	pan#
noun	pending account number	pan3
noun	pending account number	pans
noun	periodic accumulated income distribution	paid
noun	periodic review attestation form	pra
noun	person name	david
noun	person name	jeff
noun	person name	kimberly
noun	person name	naem
noun	person name	name
noun	person name	named
noun	person name	names
noun	person name	namnes
noun	personal holding company account	phc
noun	personal identification number	pin
noun	personal identification number	pins
noun	phone number	mobile
noun	phone number	number
noun	phone number	ph#
noun	plan account	plain
noun	plan sponsor	sponsor
noun	plating address	addreees
noun	plating address	plate
noun	plating address	plating
noun	plating name	plating
noun	pledged account	collateral
noun	pledged account	collatral
noun	pledged account	pledge
noun	pledged account	pledged
noun	politically exposed person	pep
noun	possible deceased client red flag	10a
noun	power of attorney	attorney
noun	power of attorney	dpoa
noun	power of attorney	pao
noun	power of attorney	poa
noun	power of attorney	poacan
noun	power of attorney	poas
noun	power of attorney	pos
noun	power of attorney form	epoas
noun	power of attorney form	mypoa
noun	power of attorney form	pao
noun	power of attorney form	ppoa
noun	power of attorney form	spoa
noun	power of attorney form	thispoa
noun	premier cash management	pcm
noun	prepayment	prepayent
noun	prepayment	prepaymet
noun	prepayment	prepeyment
noun	primary user	primaries
noun	primary user	primary
noun	primary user	theprimary
noun	principal	pri
noun	principal	prin
noun	principal account	pri
noun	prior year contribution	contr
noun	prior year contribution	contribution
noun	private wealth advisor	pwa
noun	private wealth management	pwm
noun	professional alliance group	pag
noun	prospect client	propect
noun	prospect client	prospect
noun	prospect client	prospects
noun	qualified charitable distribution	qcd
noun	qualified charitable distribution	qcds
noun	qualified disclaimer	disclaimer
noun	qualified disclaimer	disclaimers
noun	qualified domestic relations order	qdro
noun	qualified institutional investor	qib
noun	qualified plan participant agreement	ppa
noun	qualified plan participant agreement	qpp
noun	qualified plan participant agreement	qppa
noun	qualified plan participant agreement	qppq
noun	qualified plan participant agreement	qqp
noun	rebate	cashback
noun	rebate	rebates
noun	receive versus payment	rvp
noun	red flag	flag
noun	red flag	flagged
noun	red flag	flags
noun	red flag	redflag
noun	refund	ref
noun	refund	refun
noun	refund	refunded
noun	refund	refunds
noun	refund	return
noun	registered client service associate	rcsa
noun	religious organization	church
noun	request for taxpayer identification number and certification	w9f
noun	request for taxpayer identification number and certification	w9s
noun	required death distribution	rdd
noun	required death distribution	rdds
noun	required death distribution	rddstart
noun	required minimum distribution	rmd
noun	required minimum distribution	rmdcalculated
noun	required minimum distribution	rmddist
noun	required minimum distribution	rmdrolled
noun	required minimum distribution	rmds
noun	required minimum distribution	rmds!
noun	research and strategy insights	rsi
noun	reserved living & giving	reserved
noun	residual distribution	balance
noun	residual distribution	esidual
noun	residual distribution	reisudal
noun	residual distribution	remainder
noun	residual distribution	reseidual
noun	residual distribution	residual
noun	residual distribution	residuals
noun	residual distribution	resiudals
noun	restriction	limitations
noun	restriction	restrict
noun	restriction	restrictions
noun	retail banking institution account	ban
noun	retirement account	retireemnt
noun	retirement account	retirement
noun	retirement account	retm
noun	retirement plan manager account	rmp
noun	retirement plan manager account	rpm
noun	retirement plan manager account	rpms
noun	retirement plan manager agreement	rpm
noun	retirement plan manager plan account	rpm
noun	retirement plan manager subaccount	rpa
noun	retirement plan manager subaccount establishment form	retsaef
noun	retirement plan manager subaccount establishment form	sef
noun	retirement plan manager team	rpm
noun	retirement support services operations	rss
noun	retirement system	rtm
noun	return on asset	roa
noun	return on equity	roe
noun	returned stock certificate	reorg
noun	revocable living trust account	rev
noun	revocable living trust account	revoc
noun	revocable living trust account	revocable
noun	reward point	points
noun	rights of accumulation	roa
noun	rollover	rol
noun	rollover	rolled
noun	rollover	rolling
noun	rollover	rolllover
noun	rollover	rolllovers
noun	rollover	rollout
noun	rollover	rolls
noun	rollover	rollvoer
noun	rollover	rolover
noun	rollover ira account	rol
noun	rollover ira account	roll
noun	rollover ira account	rollover
noun	rollover ira account	rollovers
noun	rollover ira account	rolover
noun	roth conversion	conversion
noun	roth conversion	conversions
noun	roth distribution	contributions
noun	roth inherited ira account	cnr
noun	roth inherited remainder ira account	cne
noun	roth ira account	rot
noun	roth ira account	roth
noun	roth ira account	roths
noun	salary reduction sar-sep account	sarsep
noun	sar sep participant account	spc
noun	sar sep principal account	spl
noun	sar-sep ira plan employer adoption agreement	retsarg
noun	secondary user	secondary
noun	securities investor protection corporation insured	sipc
noun	security journal	security
noun	self-directed retirement account	sdr
noun	self-directed retirement account	sdra
noun	self-directed retirement account	sdras
noun	separately managed account	sma
noun	service portal inquiry	sri
noun	sibling	siblings
noun	signature	sig
noun	signature	sign
noun	signature	signatures
noun	signature	signaute
noun	signature	signed
noun	signature	signing
noun	signature	signiture
noun	signature	signs
noun	signature	sigs
noun	simple ira account	simple
noun	simple ira account	simples
noun	simple ira account	wimple
noun	simple ira participant account	spb
noun	simple ira plan employer adoption agreement	retsmplg
noun	simple ira principal account	spi
noun	simplified employee pension account	sep
noun	simplified employee pension account	seps
noun	simplified employee pension participant account	spa
noun	simplified employee pension principal account	spr
noun	single account	ind
noun	single account	indi
noun	single account	indiv
noun	single account	indivi
noun	single account	individ
noun	single account	individual
noun	single account	indv
noun	single account	sing
noun	single account	single
noun	single advisory contract	sac
noun	single advisory contract	sacbe
noun	single advisory contract	sacs
noun	single advisory contract	sar
noun	single advisory contract	sca
noun	single advisory contract	ssac
noun	single member limited liabilities company account	dre
noun	single member limited liabilities company account	llc
noun	single member limited liabilities company account	smllc
noun	social security number	snn
noun	social security number	ss#
noun	social security number	ssn
noun	social security number	ssn#
noun	social security number	ssns
noun	social security number card	san
noun	social security number card	sscard
noun	social security number card	ssn
noun	sole proprietorship account	sol
noun	son	sons
noun	sort name	short
noun	sort name	sortname
noun	special needs trust account	snt
noun	spousal beneficiary	wifes
noun	spousal distribution	distribution
noun	spouse	husband
noun	spouse	husbands
noun	spouse	spousal
noun	spouse	wife
noun	spouse	wifes
noun	state	cali
noun	state	california
noun	state	connecitcut
noun	state	connecticut
noun	state	d.c
noun	state	d.c.
noun	state	dallas
noun	state	delaware
noun	state	florida
noun	state	illinois
noun	state	illnois
noun	state	iowa
noun	state	iowas
noun	state	kansas
noun	state	kentucky
noun	state	louisiana
noun	state	louisianan
noun	state	maine
noun	state	maryland
noun	state	mass
noun	state	massachusetts
noun	state	michigan
noun	state	mississippi
noun	state	n.c
noun	state	nebraska
noun	state	nevada
noun	state	ohio
noun	state	oregon
noun	state	orgen
noun	state	pennsylvania
noun	state	s.c
noun	state	states
noun	state	tennessee
noun	state	texas
noun	state	vermont
noun	state	virginia
noun	state	washington
noun	state tax withholding	withhold
noun	stock plan services account	sps
noun	street name	street
noun	sub account	sub
noun	sub account	subaccount
noun	sub account	subaccounts
noun	sub account	subacct
noun	sub account	subaccts
noun	sub account	subacount
noun	sub account	subs
noun	sub trust account	subtrust
noun	subaccount establishment form	sef
noun	successor owner	successor
noun	systematic investment plan	sip
noun	systematic withdrawal plan	swp
noun	tax	taxable
noun	tax	taxation
noun	tax	taxdes
noun	tax	taxed
noun	tax	taxes
noun	tax	taxs
noun	tax enforcement - allow trading red flag	5b1
noun	tax enforcement - block all transactions red flag	5b2
noun	tax enforcement - liquidation orders only red flag	5b3
noun	tax id	atin
noun	tax id	ftin
noun	tax id	itin
noun	tax id	tax
noun	tax id	taxid
noun	tax id	tid
noun	tax id	tin
noun	tax id	tin3
noun	tax id	tins
noun	tax withholding	tax
noun	tax withholding	taxes
noun	tax withholding	taxwithholdings
noun	tax withholding	withhholding
noun	tax withholding	withhodlings
noun	tax withholding	withholding
noun	tax withholding	withholdings
noun	tax withholding	witholdings
noun	taxline team	taxline
noun	team joint account	tfa
noun	termination fee	termination
noun	testamentary trust account	tru
noun	the morgan stanley traditional ira account opening booklet	retirat
noun	the tax equity and fiscal responsibility act	tefra
noun	third party administrator	tpa
noun	third party pledge agreement	lenlaltp
noun	third party pledge and guaranty agreement	lenlalpg
noun	third party verbal authorization form	tpv
noun	third party verbal authorization form	verbal
noun	thriftsavingsplan	tsp
noun	trade	trad
noun	trade	trader
noun	trade	trades
noun	trade	trading
noun	traditional inherited remainder ira account	cnt
noun	traditional ira account	tra.d
noun	traditional ira account	trad
noun	traditional ira account	tradition
noun	traditional ira account	traditional
noun	traditional ira account	tradtional
noun	traditional ira account	transitional
noun	transfer	fts
noun	transfer	tranfer
noun	transfer	tranfser
noun	transfer	trans
noun	transfer	transf
noun	transfer	transferred
noun	transfer	transferring
noun	transfer	transfers
noun	transfer	transfewr
noun	transfer	trsnafer
noun	transfer	xfer
noun	transfer	xfger
noun	transfer	ytransfer
noun	transfer on death	t.o.d
noun	transfer on death	tod
noun	transfer on death	tods
noun	transfer on death agreement	tod
noun	transfer on death agreement	tods
noun	transfer on death beneficiary	tod
noun	treasury inflation protected securities	tips
noun	trust account	trst
noun	trust account	trsut
noun	trust account	trust
noun	trust account	trusted
noun	trust account	trustl
noun	trust account	trusts
noun	trust account	turst
noun	trust account	tust
noun	trust beneficiary	beneficiary
noun	trust beneficiary	beneficiarytrust
noun	trust beneficiary	trusts
noun	trust company account	sbg
noun	trust name	name
noun	trust name	title
noun	trust name	trusts
noun	trust name	ttle
noun	trustee	trstee
noun	trustee	trsutee
noun	trustee	truste
noun	trustee	trusteee
noun	trustee	trustees
noun	trustee	trustes
noun	trustee	trustess
noun	trustee	trutee
noun	trustee	ttee
noun	trustee	ttees
noun	trustee	ttrusee
noun	trustee	ttrustee
noun	trustee	tustee
noun	trustee certification form	trustee
noun	u.s government entities - liquidations only red flag	gov1
noun	u.s. customer due diligence rule	cdd
noun	u.s. dollar	dollar
noun	u.s. dollar	dollars
noun	u.s. dollar	usd
noun	ultimate beneficial owner	ubo
noun	ultimate beneficial owner	ubos
noun	undeliverable mail red flag	16a
noun	undeliverable mail red flag	eundleiverable
noun	undeliverable mail red flag	udl
noun	undeliverable mail red flag	undeliverable
noun	undeliverable mail red flag	underliverable
noun	unified estate request	u.e
noun	unified estate workflow	eue
noun	unified estate workflow	nue
noun	unified estate workflow	u.e
noun	unified estate workflow	u.e.
noun	unified estate workflow	ues
noun	unified estate workflow	uew
noun	unified estate workflow	uie
noun	unified estate workflow	unified
noun	unified estate workflow	unifiedestates
noun	unified managed account	uma
noun	unified managed account	umas
noun	uniform gift to minors act account	ugma
noun	uniform gift to minors act account	ugmas
noun	uniform transfer to minors act account	utam
noun	uniform transfer to minors act account	utma
noun	uniform transfer to minors act account	utmas
noun	unincorporated association account	uninc
noun	unincorporated association account	unincorporated
noun	unit investment trust	uit
noun	unrelated business income	ubi
noun	unrelated business taxable income	ubti
noun	usufruct account	usu
noun	usufruct account	usufruct
noun	usufruct account	usufructory
noun	venture capital - joint tenants by the entirety account	vje
noun	venture capital - joint tenants in common account	vjc
noun	venture capital - sole proprietorship account	vsp
noun	venture capital - unincorporated association account	vua
noun	venture capital community property account	vcp
noun	venture capital corporation account	vci
noun	venture capital living trust account	vtr
noun	venture capital llc account	vlc
noun	venture capital partnership account	vps
noun	venture capital service account	vcs
noun	verbal distribution authorization	avda
noun	verbal distribution authorization	vda
noun	verbal distribution authorization	vdas
noun	verbal distribution authorization	verbal
noun	versatile investment program account	vip
noun	versatile investment program account	vipo
noun	versatile investment program account	vips
noun	versatile investment program basic account	vipb
noun	versatile investment program basic plan account	vbp
noun	versatile investment program basic subaccount	vba
noun	versatile investment program plus account	plus
noun	versatile investment program plus account	vipp
noun	versatile investment program plus plan account	vpp
noun	versatile investment program plus subaccount	vpa
noun	versatile investment program subaccount establishment form	retvipsa
noun	vip qualified plan pre-adoption questionnaire	retqpque
noun	w-8	nnaw8
noun	w-8	w82
noun	w-8	w8rejected
noun	w-8	w8s
noun	w-8 ben	thew8ben
noun	w-8 ben	w8b
noun	w-8 ben	w8ben
noun	w-8 ben-e	w8bene
noun	w-8 imy	w8imy
noun	wealth builder account	wbd
noun	webnas	webnes
noun	wire transfer	wire
noun	wire transfer	wired
noun	wire transfer	wires
noun	wire transfer	wiring
noun	workflow	wfh
noun	workflow	workfllow
noun	workflow	workflows
noun	workflow dashboard	wfd
noun	worthless security	worthless
noun	zip code	zip
noun	zip code	zipcodenoun	required minimum distribution	zrmd
CD	ct-w4p	w4p
"""

In [82]:
#
# Keywords which should not be removed at all from input text.
#
vocab_list = ["who", "what", "where", "when", "would", "which", "how", "why", "can", "may", "will", "won't", "does", 
              "does not", "doesn't", "do", "do i", "do you", "is it", "would you", "is there", "are there", "is it so",
              "is this true" ,"to know", "is that true", "are we", "am i", "question is", "can i", "can we", "tell me", 
              "can you explain", "how ain't", "question", "answer", "questions", "answers", "ask", "can you tell", 
              'e*trade','etrade','et','etrde', "morgan staney", "ms", "ms@work"]

# containg all resource files
resources_dir_path = "/v/region/na/appl/mswm/ainlp/data/ainlp_dev/Pretrained_Models/Resources"
"""
os.listdir(resources_dir_path)
> 
['chatwords.txt',
 'contractions.json',
 'emojis.txt',
 'emoticons.json',
 'greeting_words.txt',
 'signature_words.txt',
 'special_characters.txt',
 'special_stopwords.txt',
 'stopwords.txt',
 'synonyms_noun_verb.txt']
"""

# BASIC
preprocessText = basic_preprocessing(resources_dir_path, vocab_list=vocab_list, custom=False)

# SPACY ADVANCED
preprocessText = spacy_preprocessing(resources_dir_path)


def cleaning(data, text_col, clean_text_col_name=None):
    if clean_text_col_name is None:
        clean_text_col_name =  "clean_%s" % text_col
    # add a progress tracker here...
    data[clean_text_col_name] = preprocessText.clean(data[text_col])
    return data


## SAMPLE
# df = cleaning(df, <_TEXT_COLUMN_>, <_CLEANED_TEXT_COL_NAME>)

stopwords loaded.
synonyms_noun_verb loaded.
contractions loaded.


In [83]:
df = cleaning(df, 'TEXT', 'BASIC_CLEAN_TEXT')
df = cleaning(df, 'TEXT', 'SPACY_CLEAN_TEXT')
df.head()

,TEXT,BASIC_CLEAN_TEXT,SPACY_CLEAN_TEXT
0,We have a client that wants to order a Platinu...,client want order platinum american express st...,"{'word_list': ['we', 'have', 'a', 'client', 't..."
1,how do i stop process of spousal continuation ...,how do stop process spousal continuation annuity,"{'word_list': ['how', 'do', 'i', 'stop', 'proc..."
2,can reg D forms be sent via US Mail,can reg d form send via us mail,"{'word_list': ['can', 'reg', 'd', 'forms', 'be..."
3,Is qualifying under Reg D only done digitally,qualify reg d do digitally,"{'word_list': ['is', 'qualifying', 'under', 'r..."
4,can you do an ira to roth conversion inkind,can do ira roth conversion inkind,"{'word_list': ['can', 'you', 'do', 'an', 'ira'..."


## Techniques

Methods used: 
    
1. Spacy preprocessing + TFIDF vectorization + KMeans Algorithm (iterative)
2. Basic preprocessing + TFIDF vectorization + KMeans Algorithm (iterative)
3. HAC

### Spacy preprocessing + TFIDF vectorization + KMeans Algorithm

In [130]:
#!/ms/dist/python/PROJ/core/3.7.1/bin/python

###########################################
# classes for parallel processing
###########################################
class Worker(multiprocessing.Process):
    def __init__(self, task_queue, result_queue):
        multiprocessing.Process.__init__(self)
        self.task_queue = task_queue
        self.result_queue = result_queue
    def run(self):
        proc_name = self.name
        while True:
            next_task = self.task_queue.get()
            if next_task is None:
                # Poison pill means shutdown
                # print("%s: Exiting"%proc_name)
                self.task_queue.task_done()
                break
            # print("Worker run -- %s: %s" % (proc_name, next_task))
            answer = next_task()
            self.task_queue.task_done()
            self.result_queue.put(answer)
        return

class Task(object):
    def __init__(self, task_index, split, sentence_pairs, sentences):
        self.task_index = task_index
        self.split = split
        self.sentence_pairs = sentence_pairs
        self.sentences = sentences
    def __call__(self):
        task_results = []
        for index, sentence_pair in enumerate(self.sentence_pairs):
            sentence1 = self.sentences[sentence_pair[0]]
            sentence2 = self.sentences[sentence_pair[1]]
            sentence1_length = len(sentence1)
            sentence2_length = len(sentence2)
            # compute FuzzyWuzzy similarity
            similarity = min(fuzz.ratio(self.sentences[sentence_pair[0]], self.sentences[sentence_pair[1]]),
                             fuzz.ratio(self.sentences[sentence_pair[1]], self.sentences[sentence_pair[0]]))
            task_results.append({"sentence_pair": sentence_pair, "similarity": similarity})
        # print("task %s done with %s results" % (self.task_index, len(task_results)))
        return task_results
    def __str__(self):
        return "Task index %s for %s sentences" % (self.task_index, len(self.split))
###########################################


class query_clustering():

    def __init__(self, resource_dir):
        self.stopwd_file = os.path.join(resource_dir, "stopwords.txt")
        self.synonym_file = os.path.join(resource_dir, "synonyms_noun_verb.txt")
        self.clustering_type = "noun"
        self.kmeans_rate = KMEANS_RATE
        self.cluster_k = CLUSTER_K
        self.kmeans_seed_init = KMEANS_SEED_INIT
        self.kmeans_maxiter = KMEANS_MAXITER
        self.cluster_length = CLUSTER_LEN
        self.cohesion_threshold = COHESION_THRESHOLD
        self.stopwords = []
        self.synonym_dict = {}
        self.load_stop_words()
        self.load_syn_verb_dict()
        # print input text
        self.test_mode = "no"

    def load_stop_words(self):
        with io.open(self.stopwd_file, 'r', encoding='utf-8', errors='ignore') as f:
            self.stopwords = [x.rstrip() for x in f.readlines()]

    def load_syn_verb_dict(self):

        synonym_list = []

        with io.open(self.synonym_file, 'r', encoding='utf-8', errors='ignore') as f:
            synonym_list = [x.rstrip() for x in f.readlines()]
            for line in synonym_list:
                if len(line) < 4 or re.search("^#", line):
                    continue

                syn_words = line.lower().split("\t")
                pos = syn_words[0]
                hwd = syn_words[1]
                syn_words.pop(0)
                syn_words.pop(0)

                for s in syn_words:
                    self.synonym_dict.setdefault(s, {})[pos] = hwd

    def get_pos_list(self, results):
        pos_ans = []
        word_list = []
        lemma_list = []
        ne_list = []
        start_end_list = []
        indices = results['sentences']
        for line in indices:
            tokens = line['tokens']
            for token in tokens:
                pos_syn = ""
                pos = token['pos'].lower()
                pos_ans.append(token['pos'])
                word_list.append(token['word'])
                lemma = token['lemma'].lower()

                if lemma in self.stopwords and lemma not in ['want', 'against', 'further', 'online', 'same', 'under',
                                                             'what', 'want', 'when', 'own'] \
                        or lemma in [":", "-lrb-", "-rrb-", "-lsb-", "-rsb-", "\\", '-pron-', '_', 'card num', '"'] \
                        or pos == ":" or pos == "." or re.search('^([\W]*)$]', lemma) or len(lemma) >= 30:
                    continue

                if re.search('^nn', pos):
                    pos_syn = 'noun'
                elif re.search('^v', pos):
                    pos_syn = 'verb'
                elif re.search('^adj', pos):
                    pos_syn = 'adj'

                if lemma in list(self.synonym_dict.keys()) and \
                        pos_syn in list(self.synonym_dict[lemma].keys()):
                    lemma = self.synonym_dict[lemma][pos_syn]

                lemma_list.append(lemma)

                ne_list.append(token['ner'])
                start_end_list.append(str(token['characterOffsetBegin']) + "_" + str(token['characterOffsetEnd']))
        # print(pos_ans); print(word_list); print(lemma_list); print(ne_list)
        if len(lemma_list) == 0:
            sent_lemma = "NO_NP_KEYS"
        else:
            sent_lemma = " ".join(lemma_list)

        return " ".join(word_list), sent_lemma, " ", " "

    def get_sent_lemma(self, sent):
        # allowed spacy attributes
        props_str = [{'value': 'false', 'key': 'enforceRequirements'},
                     {'value': 'json', 'key': 'outputFormat'},
                     {'value': 'normalizeSpace=false, strictTreebank3=true', 'key': 'tokenize.options'},
                     {'value': 'tokenize,ssplit,pos,lemma,ner', 'key': 'annotators'},
                     {'value': 'true', 'key': 'ssplit.eolonly'}]
        input_json = {'text': sent, 'props': props_str}
        results = run_pipeline(input_json)
        return self.get_pos_list(results)

    def run_doc_clustering(self, documents, k_val):

        k_val = int(k_val)
        vectorizer = TfidfVectorizer(use_idf=True, ngram_range=(1, 1))

        X = vectorizer.fit_transform(documents)
        terms = vectorizer.get_feature_names()
        if self.clustering_type == "fixed":
            true_k = 2000  # 1078
        elif self.clustering_type == "noun":
            true_k = int(np.sqrt(len(documents) / 2)) * self.kmeans_rate
        # true_k = 800
        elif self.clustering_type == "verb":
            true_k = int(np.sqrt(len(documents) / 2))
        else:
            print("Please set up clustering type!")

        max_iter = self.kmeans_maxiter
        # n_init = self.kmeans_seed_init

        if k_val > 0 and self.clustering_type == "noun":
            # print(len(documents)/2, k_val)
            if len(documents) / 2 < k_val:
                print("Number of input lines:", len(documents))
                print("K value provided:", k_val)
                print("Suggested value of K for the first level clustering is smaller than input lines divided by 2. Please try again!")
                print("Heusristic value of K if line number > 100:", int(np.sqrt(len(documents) / 2)) * self.kmeans_rate)
                exit()
            else:
                true_k = k_val

        if true_k == 0:
            true_k = 1

        ninit = 50
        r_num = len(documents) - 1

        model = KMeans(n_clusters=true_k, init='k-means++', max_iter=2000, random_state=r_num, n_init=ninit)
        model.fit(X)

        return model, terms, X


###########################################
# Utility functions
############################################

# Execute Spacy Pipeline
def run_pipeline(input_json):
    """
    Runs Spacy pipeline specified by params.
    param: input_json = {'text': sentence, 'props': props_str}
    return: output_json
    """
    # Get all parameters from input JSON
    text = input_json["text"]
    for prop in input_json["props"]:
        if prop["key"] == "annotators":
            operations = prop["value"].split(",")

    # Run Spacy Pipeline
    doc = spacy_model(text)
    doc_json = doc.to_json()  # Includes all info from spacy pipeline

    output_json = {}
    output_json["sentences"] = []

    # a. create necessary dictionaries
    # a.1 Extract Entity List
    entity_list = doc_json["ents"]
    # a.2 create token lib
    token_lib = {token["id"]: token for token in doc_json["tokens"]}

    # b. add sentence indices
    for i, sentence in enumerate(doc_json["sents"]):
        out_sentence = {"index": i, "line": 1, "tokens": []}
        parse = ""
        basicDependencies = []
        output_json["sentences"].append(out_sentence)

        # c. split sentences by indices, add labels (pos, ner, dep, etc.)
        for token in doc_json["tokens"]:
            if sentence["start"] <= token["start"] and token["end"] <= sentence["end"]:
                # Extract Entity label
                ner = "O"
                for entity in entity_list:
                    if entity["start"] <= token["start"] and token["end"] <= entity["end"]:
                        ner = entity["label"]
                # Extract dependency info
                dep = token["dep"]
                governor = 0 if token["head"] == token["id"] else (token["head"] + 1)  # CoreNLP index = pipeline index +1
                governorGloss = "ROOT" if token["head"] == token["id"] else text[token_lib[token["head"]]["start"]: token_lib[token["head"]]["end"]]
                dependent = token["id"] + 1
                dependentGloss = text[token["start"]:token["end"]]
                lemma = doc[token["id"]].lemma_
                # d. add dependencies
                basicDependencies.append({"dep": dep,
                                          "governor": governor,
                                          "governorGloss": governorGloss,
                                          "dependent": dependent,
                                          "dependentGloss": dependentGloss})
                # e. add tokens
                out_token = {"index": token["id"] + 1,
                             "word": dependentGloss,
                             "originalText": dependentGloss,
                             "characterOffsetBegin": token["start"],
                             "characterOffsetEnd": token["end"]}
                if "lemma" in operations:
                    out_token["lemma"] = lemma
                if "pos" in operations:
                    out_token["pos"] = token["tag"]
                if "ner" in operations:
                    out_token["ner"] = ner
                out_sentence["tokens"].append(out_token)
        if "parse" in operations:
            out_sentence["parse"] = parse
            out_sentence["basicDependencies"] = basicDependencies
            out_sentence["enhancedDependencies"] = out_sentence["basicDependencies"]
            out_sentence["enhancedPlusPlusDependencies"] = out_sentence["basicDependencies"]

    return output_json


def fileToString(filename, exclude_comments=True, keep_newline=False):
    string_result = ''
    with open(filename, mode='r') as f:
        if exclude_comments:
            for line in f:
                if not line.strip().startswith("#"):
                    if keep_newline:
                        string_result += line
                    else:
                        string_result += (line.strip() + ' ')
            return string_result
        else:
            return f.read()

def fileToJson(filename, exclude_comments=True):
    string_rep = fileToString(filename, exclude_comments=exclude_comments)
    return json.loads(string_rep)

def sanitize_text(text):
    if not isinstance(text, str):
        text = text.decode('utf-8')
    return ftfy.fix_text(text).encode('ascii', 'ignore').decode('utf-8')

def remove_duplicates(data):
    for id in list(data.keys()):
        if data[id]["keep"] == "no":
            del data[id]
    #sort data
    new_data = {}
    idx = 0
    for id, x in list(data.items()):
        new_data[idx] = data[id]
        idx += 1
    return new_data

def remove_no_keys(data):
    data_tmp = {}
    index = 0
    no_key_count = 0
    for id in list(data.keys()):
        if data[id]["sent_lemma"] != "NO_NP_KEYS":
            data_tmp[index] = data[id]
            index = index + 1
        else:
            # print("Number removed:",data[id]["sent_raw"])
            no_key_count = no_key_count + 1
    return data_tmp

def calculate_duplicate(data):
    ids = list(data.keys())

    case_threshold = 0.5

    if len(ids) == 0:
        return data
    ids.pop(0)

    for id in list(data.keys()): # 0, 1, 2, n-1
        s_index = id

        data.setdefault(id, {})["index_set"] = set()

        data[id]["index_set"].add(id)
        for id_sub in ids:
            max_rate = data[id]["case_ratio"]
            #data[id]["index_set"].add(id)
            if data[id]["count"] != 0 and data[id_sub]["count"] != 0 and \
               data[id]["sent_norm"] == data[id_sub]["sent_norm"]:

                if data[id_sub]["case_ratio"] > max_rate and data[id_sub]["case_ratio"] < case_threshold:
                    max_rate = data[id_sub]["case_ratio"]
                    s_index = id_sub

                data[id]["count"] += 1
                data[id]["index_set"].add(id_sub)
                data[id_sub]["count"] = 0
        if ids != []:
            ids.pop(0)
        if data[id]["count"] != 0:
            data[id]["index"] = s_index
    return data

def cluster_center_sent(clustering, cluster_set_by_sid, data):
    document_center = []
    sid_mapping_center = {}
    sid_tmp = 0
    sent_num_cid = len(cluster_set_by_sid)

    dup_sid_set = []
    for sid in cluster_set_by_sid:
        sid_mapping_center[sid_tmp] = sid
        if len(data[sid]['sent_lemma']) > 0:
            dup_sid_set.append(sid)
        document_center.append(data[sid]['sent_lemma'])
        sid_tmp +=1

    if len(dup_sid_set) == 1:
        seedid1 = list(dup_sid_set)[0]
        return seedid1, seedid1

    # clustering includes duplcated ones
    model_center, terms_center, X_center = clustering.run_doc_clustering(document_center, 1)
    cluster_centers_center  = model_center.cluster_centers_ # added
    DISTMIN = -9999

    values = []
    values_sid_set = []
    # remove duplicated ones
    for i in range(0,sent_num_cid):
        sid_data = sid_mapping_center[i]
        if data[sid_data]["count"] > 0:
            d_vec = X_center[i].toarray()
            #values.append(spatial.distance.euclidean(d_vec, cluster_centers_center[0,:]))
            if math.isnan(spatial.distance.cosine(d_vec, cluster_centers_center[0,:])):
                values.append(1)
            else:
                values.append(1 - spatial.distance.cosine(d_vec, cluster_centers_center[0,:]))
            #score = 1 - spatial.distance.cosine(d_vec, cluster_centers[int(cid),:]) #mainid
            values_sid_set.append(sid_data)

    seedid1_tmp = values.index(max(values))
    min_dist1 = values[seedid1_tmp]
    values[seedid1_tmp] = DISTMIN
    seedid1 = values_sid_set[ seedid1_tmp ]

    seedid2_tmp = values.index(max(values))
    min_dist2 = values[seedid2_tmp]
    seedid2 = values_sid_set[ seedid2_tmp ]
    values[seedid1_tmp] = min_dist1 # recover weight

    return seedid1, seedid2, values, values_sid_set

def cluster_cohesion_cosine(clustering, cluster_set_by_sid, data):
    document_center = []
    values = []

    for sid in cluster_set_by_sid:
        document_center.append(data[sid]['sent_lemma'])

    model_center, terms_center, X_center = clustering.run_doc_clustering(document_center, 1)
    cluster_centers_center  = model_center.cluster_centers_

    for i in range(0, len(cluster_set_by_sid)):
        d_vec = X_center[i].toarray()
        simval = spatial.distance.cosine(d_vec, cluster_centers_center[0,:])
        if math.isnan(simval):
            values.append(0)
        else:
            values.append(1 - simval)
    if len(values) ==0:
        return 0
    else:
        return sum(values) / len(values)

def pairwiseCohesionScore(data_list):
    sentences = data_list

    # get the sentence index pairs
    sentence_pairs = []
    perm = list(permutations(list(range(len(sentences))), 2))
    for (i, j) in perm:
        # remove duplicated pairs: e.g., given (i,j) and (j,i), keep only one
        if i < j:
            sentence_pairs.append((i, j))

    # Establish communication queues
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()
    # start workers by having 75% of all available cores
    num_workers = int(multiprocessing.cpu_count() * 0.60)
    # print("Creating %d workers"%num_workers)
    workers = [Worker(tasks, results) for i in range(num_workers)]
    for w in workers:
        w.start()
    # split the job based on the number of workers
    work_split = np.array_split(list(range(len(sentence_pairs))), num_workers)
    # print(">>> work split:", work_split)
    job_number = 0
    for i, split in enumerate(work_split):
        if len(split) > 0:
            task = Task(i, split, sentence_pairs[split[0]:split[-1] + 1], sentences)
            tasks.put(task)
            job_number += 1
        else:
            continue
    # print(">>> no job is given to task %s with split %s"%(i, split))

    # Add a poison pill for each worker so that they will finish when the task is complete
    for i in range(num_workers):
        tasks.put(None)

    # Wait for all of the tasks to finish
    tasks.join()

    # get results from workers
    job_results = []
    while job_number:
        # print("%s job(s) to go"%job_number)
        result = results.get()
        job_results.extend(result)
        job_number -= 1

    sorted_job_results = sorted(job_results, key=lambda x: x['sentence_pair'])
    total_pairs = len(sorted_job_results)
    similarity_sum = 0

    for x in sorted_job_results:
        index1 = x['sentence_pair'][0]
        index2 = x['sentence_pair'][1]
        similarity_sum += x['similarity']
    return similarity_sum / total_pairs

def pairwiseSimCal(data_list, cluster_set, operation):

    clusters = {}
    cluster_set.clear()

    if operation == 'copy':
        cluster_set[0] = list(range(len(data_list)))
        return
    elif operation == 'split':
        for index in range(len(data_list)):
            cluster_set[index] = [index]
        return
    sentences = data_list
    # get the sentence index pairs
    sentence_pairs = []
    perm = list(permutations(list(range(len(sentences))), 2))
    for (i, j) in perm:
        # remove duplicated pairs: e.g., given (i,j) and (j,i), keep only one
        if i < j:
            sentence_pairs.append((i, j))

    # Establish communication queues
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()

    # start workers by having 75% of all available cores
    num_workers = int(multiprocessing.cpu_count() * 0.60)
    # print("Creating %d workers"%num_workers)
    workers = [Worker(tasks, results) for i in range(num_workers)]

    for w in workers:
        w.start()

    # split the job based on the number of workers
    work_split = np.array_split(list(range(len(sentence_pairs))), num_workers)
    # print(">>> work split:", work_split)

    job_number = 0

    for i, split in enumerate(work_split):
        if len(split) > 0:
            task = Task(i, split, sentence_pairs[split[0]:split[-1] + 1], sentences)
            tasks.put(task)
            job_number += 1
        else:
            continue
    # print(">>> no job is given to task %s with split %s"%(i, split))

    # Add a poison pill for each worker so that they will finish when the task is complete
    for i in range(num_workers):
        tasks.put(None)

    # Wait for all of the tasks to finish
    tasks.join()

    # get results from workers
    job_results = []

    while job_number:
        # print("%s job(s) to go"%job_number)
        result = results.get()
        job_results.extend(result)
        job_number -= 1

    # show the combined results
    sorted_job_results = sorted(job_results, key=lambda x: x['sentence_pair'])

    similarity_results = []
    count = 0
    for x in sorted_job_results:
        index1 = x['sentence_pair'][0]
        index2 = x['sentence_pair'][1]

        sentence1 = sentences[index1]
        sentence2 = sentences[index2]

        similarity = x['similarity']
        # print("threshold_sim set for similarity computation =", threshold_sim)
        if similarity > threshold_sim:
            flag = 0
            for i in range(count):
                if i in clusters and (index1 in clusters[i] or index2 in clusters[i]):
                    clusters[i].add(index1)
                    clusters[i].add(index2)
                    flag = 1
                    break

            if flag == 0:
                clusters[count] = set()
                clusters[count].add(index1)
                clusters[count].add(index2)
                count += 1

            # print("%s\t%s\t%s -- [%s] vs. [%s]"%(index1, index2, similarity, sentence1, sentence2))
            similarity_results.append({'index1': index1, 'index2': index2, 'similarity': similarity, 'sentence1': sentence1, 'sentence2': sentence2})
        else:
            clusters[count] = set()
            clusters[count].add(index1)
            count += 1
            clusters[count] = set()
            clusters[count].add(index2)
            count += 1

    # filter and merge
    for index in list(clusters.keys()):
        if bool(clusters[index]):
            for i in range(index + 1, len(list(clusters.keys()))):
                if clusters[index].intersection(clusters[i]):
                    clusters[index] = clusters[index].union(clusters[i])
                    clusters[i] = set()

    cluster_count = 0
    for index in list(clusters.keys()):
        if bool(clusters[index]):
            cluster_count += 1
            cluster_set[index] = []
            for e in clusters[index]:
                cluster_set[index].append(e)  # sid?
############################################


###########################################
# Normal Clustering
# This was implemented by using tf and idf measure with terms. It is of two level processing: (1) top-down kmeans clustering; (2) Bottom-up
# clustering with the results from the first step. Pick up clusters with element more than 10 for clustering wiht kmeans.
# Calculate cohesion scores to final clusters as reference of the quality.
def run_query_clustering(df, cluster_k):
    """
    Run k-means clustering in two levels.
    :param: df - input data
    :param: cluster_k - if cluster_k > 0, then use this value, otherwise, calculate with heustistics
    :return: output df
    """
    start = time.time()
    
    clustering = query_clustering(resources_dir_path)

    if cohesion_threshold > 0:
        clustering.cohesion_threshold = cohesion_threshold

    data = {}
    idx = 0
    documents = []
    dup_counted_file = "no"
    db_id = 1
    dup_count = 1
    
    for input_text in df[col_TEXT].values:
        if len(input_text) < 3 or re.search("^#",input_text) or re.search("^dup_cnt",input_text.lower()):
            continue
        
        input_text = str(input_text).strip()
        input_text = sanitize_text(input_text)
        input_text = str(input_text).strip()
        input_text = re.sub('[\W]*$','',input_text)		# remove ending non-alpha-numeric chars

        if clustering.test_mode == 'yes':
            print(idx,"\t",input_text)
        filter_stopwds = True
        sent_tokens, sent_lemma, sent_verb, sent_dup = clustering.get_sent_lemma(input_text)  #all

        upper_cnt = sum(1 for c in input_text if c.isupper())
        case_ratio = float(upper_cnt) / float(len(input_text))

        data.setdefault(idx, {})["sent_lemma"] = str(sent_lemma)
        data.setdefault(idx, {})["sent_noun"] = str(sent_lemma)
        data.setdefault(idx, {})["sent_verb"] = str(sent_verb)
        data.setdefault(idx, {})["sent_tokens"] = sent_tokens
        data.setdefault(idx, {})["sent_norm"] = input_text.lower()
        data.setdefault(idx, {})["sent_raw"] = input_text
        data.setdefault(idx, {})["count"] = int(dup_count)  # from dup counted file: 1
        data.setdefault(idx, {})["case_ratio"] = case_ratio
        data.setdefault(idx, {})["clusterid"] = "no"
        data.setdefault(idx, {})["keep"] = "yes"
        data.setdefault(idx, {})["select"] = "no"
        data.setdefault(idx, {})["index"] = idx  # pointing to itself when loading with dup counted file.
        data.setdefault(idx, {})["db_id"] = db_id  # pointing to itself when loading with dup counted file.

        idx += 1
        if idx % 1000 == 0:
            print("Count of lemmatization:", idx)
            print("Time:",time.time()- start," sec")
            start = time.time()

    # remove duplicated sentences
    # data = remove_duplicates(data)
    # data = remove_no_keys(data)
    if dup_counted_file == "no":
        data = calculate_duplicate(data)

    # include all sentences for clustering: multiple same sentences in the collection can change idf values
    documents =[]
    main_docid_id_map = {}  # id in documents vs real id in input
    doc_id = 0
    for id, x in list(data.items()):
        if data[id]["count"] >= 0:
            main_docid_id_map[doc_id] = id
            documents.append(data[id]['sent_lemma'])
            doc_id += 1

    print("Calculating query clusters!")
    if len(documents) < 10:
        print("The file contains sentences less than 10! Please check  and run again!", documents)
        exit()

    ## 1st CLUSTERING  ##
    clustering.clustering_type = "noun"
    model, terms, X = clustering.run_doc_clustering(documents, cluster_k)

    cluster_labels  = model.labels_
    cluster_centers  = model.cluster_centers_

    c_s_list = [(y,x) for x,y in enumerate(cluster_labels)]

    cid_sid_counter = {}
    for cid, sid in c_s_list:
        main_sid = main_docid_id_map[sid]
        data.setdefault(main_sid, {})["clusterid"] = str(format(cid,"05d"))
        cid_sid_counter.setdefault(cid, []).append(main_sid)

    clustering.clustering_type = "noun"
    cohesionThreshold = clustering.cohesion_threshold
    for cid in sorted(cid_sid_counter.keys()):

        documents_sub =[]
        doc_id_sid_index = {}
        for sid in cid_sid_counter[cid]:
            documents_sub.append(data[sid]['sent_lemma'])

        if len(documents_sub) > clustering.cluster_length:
            cohesion_score = cluster_cohesion_cosine(clustering, cid_sid_counter[cid], data)

            #### RE-CULSTERING
            if cohesion_score < cohesionThreshold:  #### to use cohision value for reclustering
                model_sub, terms, X_sub = clustering.run_doc_clustering(documents_sub,cluster_k)
            else:
                continue
        else:
            continue

        cluster_labels_sub = model_sub.labels_
        c_s_list_sub = [(y,x) for x,y in enumerate(cluster_labels_sub)]
        for cid_sub,sid_sub in c_s_list_sub:
            # print("cid, sid:",cid_sub,sid_sub)
            sid_index = cid_sid_counter[cid][sid_sub]
            data[sid_index]['clusterid'] = data[sid_index]['clusterid'] + "_"+str(format(cid_sub,"03d"))
            # cid_sid_counter_sub.setdefault(cid_sub, []).append(sid_index)

    clustering.clustering_type = "noun"  # reset to first level

    # Output
    output_data = []
    convSeqId = 1
    cluster_set_by_sid = {}
    for id, x in list(data.items()):
        if data[id]['clusterid'] != "no":
            cid = data[id]['clusterid']
            cluster_set_by_sid.setdefault(cid, []).append(id)

    output_data.append("Cluster_ID\tText_ID\tText\tSeed_Q1\tSeed_Q2\tDist_to_Center\tCohesion\r\n")
    
    T = 0.0  # recommended: 0.4
    cid_index = 0
    cluster_set_by_sid_filtered = {}
    cluster_set_by_sid_filtered_coh_score = {}
    for cid in list(cluster_set_by_sid.keys()):
        if len(cluster_set_by_sid[cid]) > 1:
            cohesion_score = cluster_cohesion_cosine(clustering, cluster_set_by_sid[cid], data)
            if cohesion_score < T:
                for s in cluster_set_by_sid[cid]:
                    cluster_set_by_sid_filtered.setdefault(cid_index,[]).append(s)
                    cid_index += 1
            else:
                cluster_set_by_sid_filtered[cid_index] = cluster_set_by_sid[cid]
                cluster_set_by_sid_filtered_coh_score[cid_index] = cohesion_score
                cid_index += 1
        else:
            cluster_set_by_sid_filtered[cid_index] = cluster_set_by_sid[cid]
            cluster_set_by_sid_filtered_coh_score[cid_index] = 0
            cid_index += 1
    for cid in list(cluster_set_by_sid_filtered.keys()):
        sent_num_cid = len(cluster_set_by_sid_filtered[cid])
        values = []
        values_sid = []

        if sent_num_cid > 0 and len(cluster_set_by_sid_filtered[cid]) > 1:
            seedid1,seedid2, values, values_sid = cluster_center_sent(clustering, cluster_set_by_sid_filtered[cid], data)
        else:
            seedid1 = cluster_set_by_sid_filtered[cid][0]
            seedid2 = cluster_set_by_sid_filtered[cid][0]
            values.append(1)
            values_sid.append(seedid1)

        seed_q1 = data[seedid1]["sent_raw"]
        seed_q2 = data[seedid2]["sent_raw"]

        sent_count = 0
        for sid in cluster_set_by_sid_filtered[cid]:
            sent_count += data[sid]["count"]
        ch_score= cluster_set_by_sid_filtered_coh_score[cid]
        for sid in cluster_set_by_sid_filtered[cid]:
            dup_count = data[sid]['count']
            if dup_count == 0:
                continue
            else:
                sid_norm= data[sid]["index"]
                sent = data[sid_norm]['sent_raw']
            if data[sid]['db_id'] != 'none':
                db_seq_id = data[sid]['db_id']

            sim_index = values_sid.index(sid)
            dist_to_center = 1.0 - values[sim_index]

            # cid, sid, dup_count, sent, seed_q1, seed_q2, dist_to_center, cohesion_score
            output_data.append("%d\t%s\t%s\t%s\t%s\t%5.3f\t%5.3f\r\n" % (cid, sid, sent, seed_q1, seed_q2, dist_to_center, ch_score))

        cid_index += 1
    print("\nClustering process finished\n")
    return output_data

# Hybrid CLustering
# This was implemented by using tf and idf measure with terms. It is of two level processing: (1) top-down kmeans clustering; (2) Further
# clustering with the results from the first step using string similarity calculation.
def run_query_clustering_hybrid(df, cluster_k):

    clustering = query_clustering(resource_dir)
    data = {}
    idx = 0
    for input_text in df[col_TEXT].values:
        if len(input_text) < 3:
            continue
        
        input_text = input_text.rstrip()
        input_text = sanitize_text(input_text)
        input_text = re.sub(' +',' ',input_text)
        input_text = re.sub('^ +', '', input_text)
        input_text = re.sub('[\W]*$','',input_text)
        input_text = str(input_text)

        if clustering.test_mode == 'yes':
            print(idx,"\t",input_text)
        filter_stopwds = True
        sent_tokens, sent_lemma, sent_verb, sent_dup = clustering.get_sent_lemma(input_text)  #all

        upper_cnt = sum(1 for c in input_text if c.isupper())
        case_ratio = float(upper_cnt) / float(len(input_text))

        data.setdefault(idx, {})["sent_lemma"] = sent_lemma
        data.setdefault(idx, {})["sent_noun"] = sent_lemma
        data.setdefault(idx, {})["sent_dup"] = sent_dup
        data.setdefault(idx, {})["sent_verb"] = sent_verb
        data.setdefault(idx, {})["sent_tokens"] = sent_tokens
        data.setdefault(idx, {})["case_ratio"] = case_ratio
        #data.setdefault(idx, {})["sent_tokens_rev"] = sent_tokens_rev
        data.setdefault(idx, {})["sent_norm"] = input_text.lower()
        data.setdefault(idx, {})["sent_raw"] = input_text
        data.setdefault(idx, {})["count"] = 1
        #data.setdefault(idx, {})["clusterid"] = "no"
        data.setdefault(idx, {})["keep"] = "yes"
        data.setdefault(idx, {})["select"] = "no"

        idx += 1

    data = calculate_duplicate(data)

    documents =[]

    main_docid_id_map = {}  # id in documents vs real id in input
    doc_id = 0
    for id, x in list(data.items()):
        if data[id]['count'] > 0:  # select one among the dup sent
            documents.append(data[id]['sent_lemma'])
            main_docid_id_map[doc_id] = id
            doc_id += 1

    print("Calculating query clusters!")
    if len(documents) > 1:
        model, terms, X = clustering.run_doc_clustering(documents, cluster_k)
    else:
        print("There is no sentences to run clustering!")
        return
    cluster_labels  = model.labels_
    cluster_centers  = model.cluster_centers_

    c_s_list = [(y,x) for x,y in enumerate(cluster_labels)]
    cid_sid_counter = {}

    # filter out noise sentences from clusters by threshold T. Mark them as "noise"
    for cid, sid in c_s_list:
        data_sid = main_docid_id_map[sid]
        cid_sid_counter.setdefault(cid, []).append(sid)

    # collecting new updated clusters
    cluster_results = {}
    count = 0
    Cohesion_up  = 60
    Cohesion_low  = 30
    Cohesion_scores = []
    for cid in sorted(cid_sid_counter.keys()): # first level
        documents_sub =[]
        sub_docid_id_map = {} # id to data_sid mapping for selected set
        cluster_set = {}
        id_index = 0
        for sid in cid_sid_counter[cid]:
            # documents_sub.append(data[sid]['sent_tokens'].lower())
            data_sid = main_docid_id_map[sid]
            documents_sub.append(data[data_sid]['sent_raw'].lower())
            sub_docid_id_map[id_index] = data_sid
            id_index += 1

        if len(documents_sub) > 1:  # 10
            cohesion_score = pairwiseCohesionScore(documents_sub)

            if Cohesion_up > cohesion_score :
                if Cohesion_low < cohesion_score:  ### Cohesion_low < score < Cohesion_up
                    pairwiseSimCal(documents_sub, cluster_set, 'cluster') # case 1: clustering

                    for cs in cluster_set: # sub_cluster
                        doc_coh = []

                        for ch in cluster_set[cs]:
                            sid = sub_docid_id_map[ch]
                            doc_coh.append(data[sid]['sent_raw'].lower())
                        # measure cohesion score of each cluster
                        if len(doc_coh) > 1:
                            c_score = pairwiseCohesionScore(doc_coh)
                            #print("cc2:",c_score)
                            Cohesion_scores.append(c_score)

                else: ### score < Cohesion_low
                    #print("remove")
                    pairwiseSimCal(documents_sub, cluster_set, 'split')
            else: ### Cohesion_up < score
                pairwiseSimCal(documents_sub, cluster_set, 'copy') # case 2: without clustering, copy only
                #print("copy",cid)
                Cohesion_scores.append(cohesion_score)

            for k in cluster_set:
                cluster_results[count] =[]
                for c_child in cluster_set[k]:
                    sid = sub_docid_id_map[c_child]  #data_id
                    cluster_results[count].append(sid) # ??? dup, keep sid
                count += 1
        elif len(documents_sub) == 1:
            cluster_results[count] =[]
            data_sid = main_docid_id_map[ cid_sid_counter[cid][0] ]
            cluster_results[count].append(data_sid)
            count += 1
        else:
            continue

    cluster_results_final_dup_count = {}  # count

    for k in cluster_results:
        dup_count = 0
        for kk in cluster_results[k]:  #sid
            dup_count += data[kk]["count"]
        cluster_results_final_dup_count[k] = dup_count

    output_data = []
    output_data.append("Cluster ID\tSent ID\tDup_count\tQuery\tSeed_Q1\tCount\r\n")

    for cid in cluster_results:
        center_text_id = cluster_results[cid][0]
        center_text = data[center_text_id]["sent_raw"].lower()
        class_sent_count = cluster_results_final_dup_count[cid]

        for sid in cluster_results[cid]:
            dup_num = data[sid]["count"]
            raw_text = data[sid]["sent_raw"].lower()
            output_data.append("%s\t%d\t%d\t%s\t%s\t%d\r\n"%(cid, sid, dup_num, raw_text, center_text, class_sent_count) )

    # overall cluster measurement
    print("cohesion score:", sum(Cohesion_scores)/float(len(Cohesion_scores)))
    print("Clustering process finished!")
    return output_data
###########################################



###########################################
## MAIN ##
###########################################

# :: config ::
col_TEXT = 'TEXT'
    
KMEANS_RATE = 5
CLUSTER_K = 0
KMEANS_SEED_INIT = 10
KMEANS_MAXITER = 1000
CLUSTER_LEN = 20
COHESION_THRESHOLD = 0.35
CLUSTER_LEN_2nd_LEVEL = 200
CLUSTER_LEN_threshold = 2000
OUTPUT_DUP = "yes"
cohesion_threshold = 0
threshold_sim = 60
cluster_k = 0
spacy_model = nlp


def execute_clustering_method1(data, user_run_mode):
    print("\nQuery Clustering using Spacy, TFIDF and Iterative KMeans....\n")
    
    if user_run_mode == "tfidf":
        # top-down (kmeans) and bottom-up (re-clustering of clusters with more than 10 members)
        print("Running: tfidf-based system")
        output = run_query_clustering(data, 0)

    elif user_run_mode == "hybrid":
        # top-down (kmeans) and bottom-up (string distance)
        print("Running: hybrid system.")
        output = run_query_clustering_hybrid(data, 0)
    else:
        print("No run_mode provided!")
   
    # output df
    cid, tid, txt, seedq1, seedq2, dtc, cos = [], [], [], [], [], [], []
    for line in output[1:]:
        line = line.split('\t')
        cid.append(line[0])
        tid.append(line[1])
        txt.append(line[2])
        seedq1.append(line[3])
        seedq2.append(line[4])
        dtc.append(line[5])
        cos.append(line[6])
        out = pd.DataFrame({"Cluster_ID": cid,
                            "Text_ID": tid,
                            "Text": txt,
                            "Seed_Q1": seedq1,
                            "Seed_Q2": seedq2,
                            "Dist_to_Center": dtc,
                            "Cohesion": cos})
        return out
###########################################

In [131]:
output_df = execute_clustering_method1(df, 'tfidf')


Query Clustering using Spacy, TFIDF and Iterative KMeans....

Running: tfidf-based system
Calculating query clusters!

Clustering process finished



In [132]:
output_df.head()

,Cluster_ID,Text_ID,Text,Seed_Q1,Seed_Q2,Dist_to_Center,Cohesion
0,0,0,We have a client that wants to order a Platinu...,How does a client close an american express card,how do i close an account,0.570,0.517\r\n
1,0,42,how do i close an account,How does a client close an american express card,how do i close an account,0.395,0.517\r\n
2,0,86,when does cash plus get fee reversal for ameri...,How does a client close an american express card,how do i close an account,0.476,0.517\r\n
3,0,228,How does a client close an american express card,How does a client close an american express card,how do i close an account,0.265,0.517\r\n
4,0,298,can we close an account that is the primary ac...,How does a client close an american express card,how do i close an account,0.539,0.517\r\n


### Basic preprocessing + TFIDF vectorization + KMeans Algorithm

Elbow/Silhouette Analysis

Silhouette score: It is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters. The score is higher when clusters are dense and well separated.

Calinski-Harabasz Index: Higher the score , the better the performances.

In [137]:
# TF & Keras
ms.version.addpkg("absl-py", "0.9.0")
ms.version.addpkg("astunparse", "1.6.3")
ms.version.addpkg("cachetools", "4.1.0")
ms.version.addpkg("gast", "0.3.3")
ms.version.addpkg("google-auth", "1.16.1")
ms.version.addpkg("google-auth-oauthlib", "0.4.1")
ms.version.addpkg("google-pasta", "0.2.0")
ms.version.addpkg("grpcio", "1.27.2")
ms.version.addpkg("h5py", "2.10.0-threadsafe")
ms.version.addpkg("idna", "2.10")
ms.version.addpkg("importlib-metadata", "1.5.0")
ms.version.addpkg("markdown", "3.2.2")
ms.version.addpkg("oauthlib", "3.0.2")
ms.version.addpkg("opt-einsum", "3.1.0")
ms.version.addpkg("protobuf", "3.10.0")
ms.version.addpkg("pyasn1", "0.4.8")
ms.version.addpkg("pyasn1-modules", "0.2.4")
ms.version.addpkg("requests-oauthlib", "1.0.0")
ms.version.addpkg("rsa", "4.0")
ms.version.addpkg("termcolor", "1.1.0")
ms.version.addpkg("wheel", "0.33.6")
ms.version.addpkg("wrapt", "1.12.1")
# CPU -- Tensorflow --
ms.version.addpkg("tensorflow-cpu", "2.2.0")
ms.version.addpkg("tensorboard", "2.2.2")
ms.version.addpkg("tensorboard-plugin-wit", "1.7.0")
ms.version.addpkg("tensorflow-estimator", "2.2.0")
ms.version.addpkg("tensorflow-hub", "0.7.0")
# GPU -- Tensorflow --
# ms.version.addpkg("tensorflow-gpu", "2.1.0")
# ms.version.addpkg("tensorboard", "2.0.0")
# ms.version.addpkg("tensorboard-plugin-wit", "1.7.0")
# ms.version.addpkg("tensorflow-estimator","2.0.0")
# ms.version.addpkg("tensorflow-hub", "0.7.0")

import tensorflow as tf
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import CustomObjectScope
from keras.utils.np_utils import to_categorical
from keras.engine.topology import Layer
from keras import backend as K
from keras import initializers as initializers, regularizers, constraints, optimizers
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM
# from keras.layers.core import Input, Dense, Activation
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.models import Sequential, Model, load_model
import tensorflow_hub as hub
print("TENSORFLOW loaded.")

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow.python.types'

In [0]:
col_TEXT = 'BASIC_CLEAN_TEXT'


def vectorize_and_elbow_analysis(df):
    
    #### Vectorization using `USE model`
    # load USE model
    use_model = hub.KerasLayer("/v/region/na/appl/mswm/ainlp/data/ainlp_dev/Model_library/USE-Model-Large")
    
    # Vectorize processed text using USE embeddings
    X = use_model(df[col_TEXT].tolist()).numpy()
    #### Vectorization using `USE model`

    # normalize
    X_Norm = preprocessing.normalize(X)

    print("\nTotal records: ", len(X_Norm))
    print("Dimension: ", X_Norm.shape[1])
    
    # :: ELBOW ANALYSIS ::
    
    # K-means determine k
    start_time = time.time()
    
    distortions, silhouettes, harabaszs = [], [], []
    K = range(0, len(X_Norm), 20)
    for k in tqdm.tqdm(K):
        kmeanModel = KMeans(n_clusters=k, random_state=7)
        kmeanModel.fit(X_Norm)
        labels = kmeanModel.labels_
        distortions.append(sum(np.min(cdist(X_Norm, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X_Norm.shape[0])
        silhouettes.append(silhouette_score(X_Norm, labels))
        harabaszs.append(calinski_harabasz_score(X_Norm, labels))
    print("cluster analysis completed. Time (mins): ", (time.time() - start_time) / 60)

    # Plot the elbow
    plt.figure(figsize=(10, 6), dpi=80)
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k')
    plt.show()
    plt.figure(figsize=(10, 6), dpi=80)
    plt.plot(K, silhouettes, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Analysis showing the optimal k')
    plt.show()
    plt.figure(figsize=(10, 6), dpi=80)
    plt.plot(K, harabaszs, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Calinski Harabasz Score')
    plt.title('Calinski Harabasz showing the optimal k')
    plt.show()
    
    return X, X_Norm

In [0]:
X, X_Norm = vectorize_and_elbow_analysis(df)

In [139]:
def execute_clustering_method1(X_Norm, cluster_k):
    
    # Cohesion Score
    def calculate_cohesion(cluster_id):
        # cluster_id and coressponding members (docs belonging to cid)
        docs = df[df['Cluster_ID'] == cluster_id][col_TEXT].tolist()

        score = 0
        if len(docs) > 2:
            vect = TfidfVectorizer(use_idf=True, ngram_range=(1,1))
            docs_norm = vect.fit_transform(docs)
            k_value = int(np.sqrt(len(docs)/2))
            cluster_model = KMeans(n_clusters=k_value, random_state=7)
            cluster_model.fit(docs_norm)
            cluster_centers_center  = cluster_model.cluster_centers_
            # cohesion scores
            values = []
            for i in range(0, len(docs)):
                d_vec = docs_norm[i].toarray()
                simval = spatial.distance.cosine(d_vec, cluster_centers_center[0,:])
                if math.isnan(simval):
                    values.append(0)
                else:
                    values.append(1 - simval)
            if len(values)==0:
                score=0
            else:
                score=sum(values)/len(values)
        return score
    
    
    # KMeans...
    # based on elbow and silhoutte score, choosing k=3000 clusters
    # final model with chosen k
    CHOSEN_K = cluster_k
    kmeanModel = KMeans(n_clusters=CHOSEN_K, random_state=7)
    kmeanModel.fit(X_Norm)

    ## Attributes ##

    # Cluster IDs
    df['Cluster_ID'] = kmeanModel.predict(X_Norm)

    # Calculating distances (squared distance) to cluster center
    X_dist = kmeanModel.transform(X_Norm)**2
    distances = []
    for dist_to_center in X_dist:
        # cluster_id
        closeset_centroid = np.argmin(dist_to_center)
        # distance to centorid
        dist = dist_to_center[closeset_centroid]
        distances.append(dist)
    df['dist_to_center'] = distances

    # init Cluster: Cohesion dict
    cluster_id_cohesion = dict(zip(sorted(df['Cluster_ID'].unique()), [0]*len(df['Cluster_ID'].unique())))
    for cid in tqdm.tqdm(cluster_id_cohesion.keys()):
        cluster_id_cohesion[cid] = calculate_cohesion(cid)
    df['Cohesion'] = df['Cluster_ID'].apply(lambda x: cluster_id_cohesion[x])
    
    # Count per cluster
    cluster_id_count = pd.Series(df['Cluster_ID'].value_counts().values, index=df['Cluster_ID'].value_counts().index).to_dict()
    df['Count'] = df['Cluster_ID'].apply(lambda x: cluster_id_count[x])
    
    # Top 2 Canidates
    cluster_grouped = df.sort_values(by=['Cluster_ID', 'dist_to_center']).groupby(by=['Cluster_ID'])['TEXT'].apply(lambda x: list(x)[:2])
    cluster_candidates = pd.Series(cluster_grouped.values, index=cluster_grouped.index).to_dict()
    df['Candidates'] = df['Cluster_ID'].apply(lambda x: cluster_candidates[x])
    
    return df

In [0]:
output_df = execute_clustering_method1(X_Norm, cluster_k = 80)

In [0]:
output_df.head()

In [0]:
# # Visualize

# Display_records = 10

# x = df.Source.tolist()[:Display_records]
# vectors = X_Norm[:Display_records]
# kmean_indices = kmeanModel.predict(vectors)

# pca = PCA(n_components=2)
# scatter_plot_points = pca.fit_transform(vectors)
# x_axis = [o[0] for o in scatter_plot_points]
# y_axis = [o[1] for o in scatter_plot_points]
# fig, ax = plt.subplots(figsize=(20, 10))
# ax.scatter(x_axis, y_axis, c=['r'])
# for i, txt in enumerate(x):
#     ax.annotate(txt, (x_axis[i], y_axis[i]))

### HAC Algorithm

- Spacy pre-processing
- USE embeddings vectorization
- HAC

#### Plotting some dedrograms

From the dendrogram plot, find a horizontal rectangle with max-height that does not cross any horizontal vertical dendrogram line.

In [0]:
col_TEXT = 'BASIC_CLEAN_TEXT'


# dendrogram
def detailed_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)
    plt.figure(figsize=(15,10))
    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

def vectorize_and_dendrogram_analysis(df):
    
    #### Vectorization using `USE model`
    # load USE model
    use_model = hub.KerasLayer("/v/region/na/appl/mswm/ainlp/data/ainlp_dev/Model_library/USE-Model-Large")
    
    # Vectorize processed text using USE embeddings
    X = use_model(df[col_TEXT].tolist()).numpy()
    #### Vectorization using `USE model`

    # normalize
    X_Norm = preprocessing.normalize(X)

    print("\nTotal records: ", len(X_Norm))
    print("Dimension: ", X_Norm.shape[1])
    
    # ward linkage
    linked_array = ward(X_Norm)

    detailed_dendrogram(
        linked_array,
        truncate_mode='lastp',
        p=30,
        leaf_rotation=90.,
        leaf_font_size=10.,
        show_contracted=True,
        annotate_above=0.50,
        max_d=1.0)

    plt.show()

    return X, X_Norm, linked_array

In [608]:
X, X_Norm, linked_array = vectorize_and_dendrogram_analysis(df)

In [900]:
# Get number of clusters directly from dendrogram using distance threshold

distance_threshold = 1.0

clusters = fcluster(linked_array, distance_threshold, criterion='distance')

print("Number of clusters cutting threshold distance: ", len(np.unique(clusters)))

Number of clusters cutting threshold distance:  6031


In [898]:
# Another Dedrogram

MAX_COPHENETIC_DIST = max(linked_array[:,2]) * 0.10 # max distance between points to be considered together. can be tuned.

fig, ax = plt.subplots(figsize=(15, 80)) # set size
ax = dendrogram(linked_array, orientation="right", color_threshold=MAX_COPHENETIC_DIST, leaf_font_size=4,
                labels=dissimilar_df['Source'].tolist())

plt.tick_params(axis= 'x', which='both',  bottom='off', top='off',labelbottom='off')
# plt.tight_layout()
plt.savefig('ward_clusters_all.png', dpi=300)
plt.show()

#### Number of optimal clusters 

In [74]:
CHOSEN_CLUSTERS_HAC = 85

In [75]:
hacModel = AgglomerativeClustering(n_clusters=CHOSEN_CLUSTERS_HAC, affinity='euclidean', linkage='ward')
hacModel.fit(X_Norm)

AgglomerativeClustering(n_clusters=5829)

No way to get distances as algorithm loses it when merging to final cluster

In [78]:
## Attributes for HAC ##

# Cluster IDs
df['Cluster_ID'] = c['HAC_Cluster_ID'] #hacModel.labels_

# no distance function in HAC
df['dist_to_center'] = np.nan

# no cohesion values
df['Cohesion'] = np.nan

# Count per cluster
cluster_id_count = pd.Series(df['Cluster_ID'].value_counts().values, index=df['Cluster_ID'].value_counts().index).to_dict()
df['Count'] = df['Cluster_ID'].apply(lambda x: cluster_id_count[x])

### Sentence-Bert Iterative KMeans

- In different notebook